In [20]:
# --- Securely store keys (run once per session) ---
import os

os.environ["BINANCE_API_KEY"] = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
os.environ["BINANCE_API_SECRET"] = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"

In [21]:
# ============================================================
# 🟢 OPENING RANGE BREAKOUT (ORB) STRATEGY — BINANCE LIVE (6-STAGE TARGETS)
# ============================================================

import pandas as pd
import datetime as dt
import pytz
from binance.client import Client

# ============================================================
# ⚙️ CONFIGURATION
# ============================================================

CONFIG = {
    "symbol": "BTCUSDT",
    "interval": "1m",                     # 1-min data for real-time accuracy
    "trade_value": 200,
    "opening_range_mins": 30,
    "or_start_time": "05:30",             # 05:30–06:00 IST
    "session_reset_hour": 5,              # reset at 5 AM IST
    "binance_api_key": "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM",
    "binance_api_secret": "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"
}

IST = pytz.timezone("Asia/Kolkata")

# ============================================================
# 📈 FETCH DATA FROM BINANCE (robust)
# ============================================================

def fetch_intraday_data_enough(symbol, interval, session_start_ist, opening_range_mins=30, buffer_mins=120):
    """Fetch enough minutes so that the 05:30–06:00 IST range is included."""
    client = Client(CONFIG["binance_api_key"], CONFIG["binance_api_secret"])

    session_start_utc = session_start_ist.astimezone(pytz.UTC)
    now_utc = dt.datetime.now(pytz.UTC)

    minutes_needed = int((now_utc - session_start_utc).total_seconds() / 60) + opening_range_mins + buffer_mins
    minutes_needed = max(minutes_needed, opening_range_mins + buffer_mins)
    minutes_needed = min(minutes_needed, 10080)  # cap at 7 days

    klines = client.get_historical_klines(symbol, interval, f"{minutes_needed} minute ago UTC")

    df = pd.DataFrame(klines, columns=[
        "Open time", "Open", "High", "Low", "Close", "Volume",
        "Close time", "Quote asset volume", "Number of trades",
        "Taker buy base", "Taker buy quote", "Ignore"
    ])
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time").tz_convert(IST)
    df = df[["Open", "High", "Low", "Close", "Volume"]].astype(float)

    print(f"✅ {len(df)} candles fetched ({minutes_needed} mins). Data covers {df.index[0]} → {df.index[-1]} IST")
    return df


def get_opening_range_robust(df, desired_start_time="05:30", opening_range_mins=30):
    """Compute OR for today's 05:30–06:00 IST; fallback if data missing."""
    now_ist = dt.datetime.now(IST)
    today = now_ist.date()
    desired_time = dt.datetime.strptime(desired_start_time, "%H:%M").time()
    desired_start = IST.localize(dt.datetime.combine(today, desired_time))
    desired_end = desired_start + dt.timedelta(minutes=opening_range_mins)

    or_df = df[(df.index >= desired_start) & (df.index < desired_end)]

    if not or_df.empty:
        start_used, end_used = desired_start, desired_end
    else:
        end_used = df.index[-1]
        start_used = end_used - dt.timedelta(minutes=opening_range_mins)
        or_df = df[(df.index >= start_used) & (df.index < end_used)]
        print(f"⚠️ Desired OR 05:30–06:00 IST not found — using fallback {start_used.strftime('%H:%M')}–{end_used.strftime('%H:%M')} IST")

    if or_df.empty:
        raise ValueError("Opening range window empty — fetch more data.")

    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2
    return or_high, or_low, or_mid, start_used, end_used

# ============================================================
# 💹 TRADE SIMULATION (6-STAGE PROFIT BOOKING)
# ============================================================

def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None

    range_size = or_high - or_low
    X = range_size / 2

    fractions = [0.15, 0.15, 0.20, 0.25, 0.15, 0.10]
    multipliers = [2, 4, 6, 8, 10]

    for i in range(1, len(df)):
        price = df["Close"].iloc[i]
        time_ = df.index[i]

        # ENTRY LOGIC
        if open_trade is None:
            if price > or_high:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "BUY", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty, "remaining_qty": trade_qty,
                    "booked": [0]*6, "targets": [entry + (X*m) for m in multipliers]
                }
                print(f"[{time_}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                continue

            if price < or_low:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "SELL", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty, "remaining_qty": trade_qty,
                    "booked": [0]*6, "targets": [entry - (X*m) for m in multipliers]
                }
                print(f"[{time_}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                continue

        # EXIT MANAGEMENT
        if open_trade:
            t = open_trade
            side = t["side"]

            # Stoploss
            if side == "BUY" and price <= t["sl"]:
                pnl = (price - t["entry_price"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": price, "exit_time": time_, "reason": "SL", "pnl": pnl})
                print(f"[{time_}] | ❌ BUY Stoploss hit @ {price:.2f}")
                open_trade = None
                continue
            if side == "SELL" and price >= t["sl"]:
                pnl = (t["entry_price"] - price) * t["remaining_qty"]
                trades.append({**t, "exit_price": price, "exit_time": time_, "reason": "SL", "pnl": pnl})
                print(f"[{time_}] | ❌ SELL Stoploss hit @ {price:.2f}")
                open_trade = None
                continue

            # Target booking
            for idx in range(5):
                if t["booked"][idx] == 0:
                    if side == "BUY" and price >= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        pnl = (t["targets"][idx] - t["entry_price"]) * book_qty
                        t["remaining_qty"] -= book_qty
                        t["booked"][idx] = 1
                        trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_,
                                       "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                        print(f"[{time_}] | 🎯 BUY TP{idx+1} hit @ {t['targets'][idx]:.2f} | {fractions[idx]*100:.0f}% booked | Remaining {t['remaining_qty']:.5f}")

                    elif side == "SELL" and price <= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        pnl = (t["entry_price"] - t["targets"][idx]) * book_qty
                        t["remaining_qty"] -= book_qty
                        t["booked"][idx] = 1
                        trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_,
                                       "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                        print(f"[{time_}] | 🎯 SELL TP{idx+1} hit @ {t['targets'][idx]:.2f} | {fractions[idx]*100:.0f}% booked | Remaining {t['remaining_qty']:.5f}")

            # End of day close
            if i == len(df)-1 and t["remaining_qty"] > 0:
                pnl = (price - t["entry_price"]) * t["remaining_qty"] if side == "BUY" else (t["entry_price"] - price) * t["remaining_qty"]
                trades.append({**t, "exit_price": price, "exit_time": time_, "reason": "TP6_EOD", "pnl": pnl, "qty": t["remaining_qty"]})
                print(f"[{time_}] | ⏰ EOD close | {side} @ {price:.2f} | Remaining {t['remaining_qty']:.5f}")
                open_trade = None

    return trades, open_trade

# ============================================================
# 📋 PRINT SUMMARY
# ============================================================

def print_closed_trades(trades):
    if len(trades) == 0:
        print("No closed trades yet.\n")
        return
    print("\n📜 Closed trades (today):")
    headers = ["side", "entry_price", "entry_time", "exit_price", "exit_time", "reason", "pnl", "qty"]
    print(" | ".join([f"{h:>10}" for h in headers]))
    for t in trades:
        print(" | ".join([
            f"{t['side']:>6}",
            f"{t['entry_price']:>10.2f}",
            f"{t['entry_time'].strftime('%Y-%m-%d %H:%M:%S'):>19}",
            f"{t['exit_price']:>10.2f}",
            f"{t['exit_time'].strftime('%Y-%m-%d %H:%M:%S'):>19}",
            f"{t['reason']:>7}",
            f"{t['pnl']:>8.2f}",
            f"{t['qty']:>8.5f}"
        ]))

# ============================================================
# ▶️ MAIN STRATEGY RUN
# ============================================================

def run_orb_strategy():
    now_ist = dt.datetime.now(IST)
    today = now_ist.date()
    session_or_start = IST.localize(dt.datetime.combine(today, dt.datetime.strptime(CONFIG["or_start_time"], "%H:%M").time()))

    df = fetch_intraday_data_enough(CONFIG["symbol"], CONFIG["interval"], session_or_start, opening_range_mins=CONFIG["opening_range_mins"], buffer_mins=180)
    or_high, or_low, or_mid, or_start_used, or_end_used = get_opening_range_robust(df, desired_start_time=CONFIG["or_start_time"], opening_range_mins=CONFIG["opening_range_mins"])

    print(f"\n📊 Opening Range:")
    print(f"  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f} | Window {or_start_used.strftime('%H:%M')}–{or_end_used.strftime('%H:%M')} IST")

    trade_qty = round(CONFIG["trade_value"] / df["Close"].iloc[-1], 5)
    trades_today, open_trade = reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty)

    total_pnl = sum(t["pnl"] for t in trades_today)
    print_closed_trades(trades_today)
    print(f"\n💰 Closed PnL: {total_pnl:.2f} USDT\n")

    if open_trade:
        print("🔔 Open trade:")
        print(f"  Side: {open_trade['side']} | Entry: {open_trade['entry_price']:.2f} | SL: {open_trade['sl']:.2f} | Remaining Qty: {open_trade['remaining_qty']:.5f}")
    else:
        print("No open trades currently.\n")

# ============================================================
# 🚀 RUN
# ============================================================

if __name__ == "__main__":
    run_orb_strategy()


✅ 569 candles fetched (569 mins). Data covers 2025-11-01 02:01:00+05:30 → 2025-11-01 11:29:00+05:30 IST

📊 Opening Range:
  OR High 109639.64 | Low 109394.81 | Mid 109517.23 | Window 05:30–06:00 IST
[2025-11-01 02:15:00+05:30] | 📈 BUY triggered @ 109755.74 | SL 109517.23
[2025-11-01 02:22:00+05:30] | ❌ BUY Stoploss hit @ 109473.86
[2025-11-01 02:44:00+05:30] | 📉 SELL triggered @ 109338.96 | SL 109517.23
[2025-11-01 02:46:00+05:30] | ❌ SELL Stoploss hit @ 109525.18
[2025-11-01 02:53:00+05:30] | 📈 BUY triggered @ 109644.93 | SL 109517.23
[2025-11-01 03:01:00+05:30] | ❌ BUY Stoploss hit @ 109451.11
[2025-11-01 03:06:00+05:30] | 📈 BUY triggered @ 109748.78 | SL 109517.23
[2025-11-01 05:32:00+05:30] | ❌ BUY Stoploss hit @ 109470.93
[2025-11-01 06:15:00+05:30] | 📈 BUY triggered @ 109650.32 | SL 109517.23
[2025-11-01 06:59:00+05:30] | 🎯 BUY TP1 hit @ 109895.15 | 15% booked | Remaining 0.00155
[2025-11-01 08:05:00+05:30] | ❌ BUY Stoploss hit @ 109494.11
[2025-11-01 08:15:00+05:30] | 📈 BUY trig

In [22]:
# ============================================================
# 🟢 OPENING RANGE BREAKOUT (ORB) STRATEGY — BINANCE VERSION (FINAL + EOD FIX)
# ============================================================

import pandas as pd
import datetime as dt
import pytz
from binance.client import Client
import time

# ============================================================
# ⚙️ CONFIGURATION
# ============================================================

CONFIG = {
    "symbol": "BTCUSDT",               # Trading pair
    "interval": "1m",                  # Candle interval
    "opening_range_mins": 30,          # ORB duration (5:30–6:00 AM IST)
    "trade_value": 200,                # Trade size in USDT
    "session_reset_hour": 5,           # Reset at 5 AM IST
    "binance_api_key": "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM",
    "binance_api_secret": "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"
}

IST = pytz.timezone("Asia/Kolkata")

# ============================================================
# 📈 FETCH DATA FROM BINANCE
# ============================================================

def fetch_intraday_data(symbol, interval, days=2):
    client = Client(CONFIG["binance_api_key"], CONFIG["binance_api_secret"])
    klines = client.get_historical_klines(symbol, interval, f"{days} day ago UTC")
    df = pd.DataFrame(klines, columns=[
        "Open time", "Open", "High", "Low", "Close", "Volume",
        "Close time", "Quote asset volume", "Number of trades",
        "Taker buy base", "Taker buy quote", "Ignore"
    ])
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time").tz_convert(IST)
    df = df[["Open", "High", "Low", "Close", "Volume"]].astype(float)
    return df

# ============================================================
# 🕔 SESSION HANDLING
# ============================================================

def get_session_start(now=None):
    now = now or dt.datetime.now(IST)
    # Session start is 5:30 AM IST of the session day
    session_start = now.replace(hour=5, minute=30, second=0, microsecond=0)
    if now.time() < dt.time(5, 30):
        session_start -= dt.timedelta(days=1)
    return session_start

# ============================================================
# 🎯 OPENING RANGE (5:30–6:00 AM IST)
# ============================================================

def get_opening_range(df, session_start):
    start = session_start
    end = start + dt.timedelta(minutes=CONFIG["opening_range_mins"])
    or_df = df[(df.index >= start) & (df.index < end)]
    if or_df.empty:
        raise ValueError("Opening range window empty.")
    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2
    return or_high, or_low, or_mid

# ============================================================
# 💹 TRADE LOGIC WITH PARTIAL TARGETS & FIXED MID STOPLOSS
#    — EOD close now only triggers when the candle timestamp is exactly 05:00 IST
# ============================================================

def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None

    range_size = or_high - or_low
    X = range_size / 2

    fractions = [0.15, 0.15, 0.20, 0.25, 0.15, 0.10]
    multipliers = [2, 4, 6, 8, 10]

    for i in range(1, len(df)):
        price = df["Close"].iloc[i]
        time_ = df.index[i]

        # ==========================
        # ENTRY CHECK: only after 5-min candle completion
        # (i.e., consider only timestamps where minute % 5 == 0)
        # ==========================
        if time_.minute % 5 != 0:
            continue

        # ==========================
        # ENTRY CONDITIONS
        # ==========================
        if open_trade is None:
            # BUY Breakout
            if price > or_high:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "BUY", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6,
                    "targets": [entry + (X*m) for m in multipliers]
                }
                print(f"[{time_}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                continue

            # SELL Breakdown
            if price < or_low:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "SELL", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6,
                    "targets": [entry - (X*m) for m in multipliers]
                }
                print(f"[{time_}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                continue

        # ==========================
        # EXIT MANAGEMENT
        # ==========================
        if open_trade:
            t = open_trade
            side = t["side"]

            # 🧨 STOPLOSS — Exit at OR MID (remaining_qty only)
            if side == "BUY" and price <= t["sl"]:
                sl_price = t["sl"]
                book_qty = t["remaining_qty"]
                if book_qty > 0:
                    pnl = (sl_price - t["entry_price"]) * book_qty
                    trades.append({
                        **t, "exit_price": sl_price, "exit_time": time_,
                        "reason": "SL", "pnl": pnl, "qty": book_qty
                    })
                    print(f"[{time_}] | ❌ BUY Stoploss hit @ {sl_price:.2f} | Closed remaining {book_qty:.5f}")
                open_trade = None
                continue

            if side == "SELL" and price >= t["sl"]:
                sl_price = t["sl"]
                book_qty = t["remaining_qty"]
                if book_qty > 0:
                    pnl = (t["entry_price"] - sl_price) * book_qty
                    trades.append({
                        **t, "exit_price": sl_price, "exit_time": time_,
                        "reason": "SL", "pnl": pnl, "qty": book_qty
                    })
                    print(f"[{time_}] | ❌ SELL Stoploss hit @ {sl_price:.2f} | Closed remaining {book_qty:.5f}")
                open_trade = None
                continue

            # 🎯 TARGET BOOKINGS (TP1–TP5)
            for idx in range(5):
                if t["booked"][idx] == 0:
                    if side == "BUY" and price >= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["targets"][idx] - t["entry_price"]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({
                                **t, "exit_price": t["targets"][idx], "exit_time": time_,
                                "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty
                            })
                            print(f"[{time_}] | 🎯 BUY TP{idx+1} hit @ {t['targets'][idx]:.2f} | {fractions[idx]*100:.0f}% booked | Remaining {t['remaining_qty']:.5f}")

                    elif side == "SELL" and price <= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["entry_price"] - t["targets"][idx]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({
                                **t, "exit_price": t["targets"][idx], "exit_time": time_,
                                "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty
                            })
                            print(f"[{time_}] | 🎯 SELL TP{idx+1} hit @ {t['targets'][idx]:.2f} | {fractions[idx]*100:.0f}% booked | Remaining {t['remaining_qty']:.5f}")

            # ⏰ EOD CLOSE — only when the timestamp is *exactly* 05:00 IST
            # This avoids closing on "any time after 05:00" (fixes earlier bug)
            if time_.time() == dt.time(CONFIG["session_reset_hour"], 0) and t["remaining_qty"] > 0:
                book_qty = t["remaining_qty"]
                if book_qty > 0:
                    # close remaining at OR mid (sl is OR mid)
                    sl_price = t["sl"]
                    pnl = (sl_price - t["entry_price"]) * book_qty if side == "BUY" else (t["entry_price"] - sl_price) * book_qty
                    trades.append({
                        **t, "exit_price": sl_price, "exit_time": time_,
                        "reason": "EOD", "pnl": pnl, "qty": book_qty
                    })
                    print(f"[{time_}] | ⏰ EOD Close | {side} @ {sl_price:.2f} | Remaining {book_qty:.5f}")
                open_trade = None

    return trades, open_trade

# ============================================================
# 📋 PRINT CLOSED TRADES SUMMARY
# ============================================================

def print_closed_trades(trades):
    if len(trades) == 0:
        print("No closed trades yet.\n")
        return

    print("\n📜 Closed trades (today):")
    headers = ["side", "entry_price", "entry_time", "exit_price", "exit_time", "reason", "pnl", "qty"]
    print(" | ".join([f"{h:>12}" for h in headers]))
    for t in trades:
        # when trades were created in reconstruct_trades_after_or they include exit_time as a tz-aware Timestamp
        # ensure formatting works for both pandas Timestamp and python datetime
        exit_time = t["exit_time"]
        entry_time = t["entry_time"]
        if hasattr(entry_time, "strftime"):
            entry_str = entry_time.strftime('%Y-%m-%d %H:%M:%S')
        else:
            entry_str = str(entry_time)
        if hasattr(exit_time, "strftime"):
            exit_str = exit_time.strftime('%Y-%m-%d %H:%M:%S')
        else:
            exit_str = str(exit_time)

        print(" | ".join([
            f"{t['side']:>6}",
            f"{t['entry_price']:>10.2f}",
            f"{entry_str:>19}",
            f"{t['exit_price']:>10.2f}",
            f"{exit_str:>19}",
            f"{t['reason']:>8}",
            f"{t['pnl']:>8.2f}",
            f"{t['qty']:>8.5f}"
        ]))

# ============================================================
# ▶️ MAIN STRATEGY EXECUTION
# ============================================================

def run_orb_strategy():
    # fetch data and build today's session slice
    df = fetch_intraday_data(CONFIG["symbol"], CONFIG["interval"], days=2)
    session_start = get_session_start()
    df_today = df[df.index >= session_start]

    or_high, or_low, or_mid = get_opening_range(df_today, session_start)
    trade_qty = round(CONFIG["trade_value"] / df_today["Close"].iloc[-1], 5)

    print(f"\n📊 Opening Range:")
    print(f"  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}\n")

    trades_today, open_trade = reconstruct_trades_after_or(df_today, or_high, or_low, or_mid, trade_qty)

    total_pnl = sum(t["pnl"] for t in trades_today)
    print_closed_trades(trades_today)
    print(f"\n💰 Closed PnL: {total_pnl:.2f} USDT\n")

    if open_trade:
        print("🔔 Open trade restored:")
        print(f"  Side: {open_trade['side']} | Entry: {open_trade['entry_price']:.2f} | SL: {open_trade['sl']:.2f}")
        print(f"  Remaining Qty: {open_trade['remaining_qty']:.5f}")
    else:
        print("No open trades currently.\n")

# ============================================================
# 🚀 RUN
# ============================================================

if __name__ == "__main__":
    run_orb_strategy()



📊 Opening Range:
  OR High 109639.64 | Low 109394.81 | Mid 109517.23

[2025-11-01 06:15:00+05:30] | 📈 BUY triggered @ 109650.32 | SL 109517.23
[2025-11-01 07:00:00+05:30] | 🎯 BUY TP1 hit @ 109895.15 | 15% booked | Remaining 0.00155
[2025-11-01 08:05:00+05:30] | ❌ BUY Stoploss hit @ 109517.23 | Closed remaining 0.00155
[2025-11-01 08:15:00+05:30] | 📈 BUY triggered @ 109738.58 | SL 109517.23
[2025-11-01 08:50:00+05:30] | 🎯 BUY TP1 hit @ 109983.41 | 15% booked | Remaining 0.00155
[2025-11-01 09:30:00+05:30] | 🎯 BUY TP2 hit @ 110228.24 | 15% booked | Remaining 0.00128

📜 Closed trades (today):
        side |  entry_price |   entry_time |   exit_price |    exit_time |       reason |          pnl |          qty
   BUY |  109650.32 | 2025-11-01 06:15:00 |  109895.15 | 2025-11-01 07:00:00 |      TP1 |     0.07 |  0.00027
   BUY |  109650.32 | 2025-11-01 06:15:00 |  109517.23 | 2025-11-01 08:05:00 |       SL |    -0.21 |  0.00155
   BUY |  109738.58 | 2025-11-01 08:15:00 |  109983.41 | 2025-11

In [23]:
"binance_api_key": "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM",
    "binance_api_secret": "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"


SyntaxError: illegal target for annotation (3031136411.py, line 1)

In [ ]:
# ============================================================
# ⚡ OPENING RANGE BREAKOUT (ORB) STRATEGY — BINANCE VERSION (FINAL + EOD FIX + LEVERAGE + TARGET PRINT)
# ============================================================

import pandas as pd
import datetime as dt
import pytz
from binance.client import Client

# ============================================================
# ⚙️ CONFIGURATION
# ============================================================

CONFIG = {
    "symbol": "BTCUSDT",               # Trading pair
    "interval": "1m",                  # Candle interval
    "opening_range_mins": 30,          # ORB duration (5:30–6:00 AM IST)
    "trade_value": 200,                # Trade size in USDT
    "leverage": 20,                    # 💥 Leverage multiplier
    "session_reset_hour": 5,           # Reset at 5 AM IST
    "binance_api_key": "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM",
    "binance_api_secret": "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"
}

IST = pytz.timezone("Asia/Kolkata")

# ============================================================
# 📈 FETCH DATA FROM BINANCE
# ============================================================

def fetch_intraday_data(symbol, interval, days=2):
    client = Client(CONFIG["binance_api_key"], CONFIG["binance_api_secret"])
    klines = client.get_historical_klines(symbol, interval, f"{days} day ago UTC")
    df = pd.DataFrame(klines, columns=[
        "Open time", "Open", "High", "Low", "Close", "Volume",
        "Close time", "Quote asset volume", "Number of trades",
        "Taker buy base", "Taker buy quote", "Ignore"
    ])
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time").tz_convert(IST)
    df = df[["Open", "High", "Low", "Close", "Volume"]].astype(float)
    return df

# ============================================================
# 🕔 SESSION HANDLING
# ============================================================

def get_session_start(now=None):
    now = now or dt.datetime.now(IST)
    session_start = now.replace(hour=5, minute=30, second=0, microsecond=0)
    if now.time() < dt.time(5, 30):
        session_start -= dt.timedelta(days=1)
    return session_start

# ============================================================
# 🎯 OPENING RANGE (5:30–6:00 AM IST)
# ============================================================

def get_opening_range(df, session_start):
    start = session_start
    end = start + dt.timedelta(minutes=CONFIG["opening_range_mins"])
    or_df = df[(df.index >= start) & (df.index < end)]
    if or_df.empty:
        raise ValueError("Opening range window empty.")
    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2
    return or_high, or_low, or_mid

# ============================================================
# 💹 TRADE LOGIC WITH PARTIAL TARGETS, LEVERAGE, AND EOD FIX
# ============================================================

def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None

    range_size = or_high - or_low
    X = range_size / 2

    fractions = [0.15, 0.15, 0.20, 0.25, 0.15, 0.10]
    multipliers = [2, 4, 6, 8, 10]

    for i in range(1, len(df)):
        price = df["Close"].iloc[i]
        time_ = df.index[i]

        # process only completed 5-min candles
        if time_.minute % 5 != 0:
            continue

        # ==========================
        # ENTRY CONDITIONS
        # ==========================
        if open_trade is None:
            # BUY Breakout
            if price > or_high:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "BUY", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6,
                    "targets": [entry + (X*m) for m in multipliers]
                }
                print(f"[{time_}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j, t in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {t:.2f}")
                continue

            # SELL Breakdown
            if price < or_low:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "SELL", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6,
                    "targets": [entry - (X*m) for m in multipliers]
                }
                print(f"[{time_}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j, t in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {t:.2f}")
                continue

        # ==========================
        # EXIT MANAGEMENT
        # ==========================
        if open_trade:
            t = open_trade
            side = t["side"]

            # 🧨 STOPLOSS — Exit at OR MID
            if side == "BUY" and price <= t["sl"]:
                sl_price = t["sl"]
                book_qty = t["remaining_qty"]
                if book_qty > 0:
                    pnl = (sl_price - t["entry_price"]) * book_qty
                    trades.append({**t, "exit_price": sl_price, "exit_time": time_,
                                   "reason": "SL", "pnl": pnl, "qty": book_qty})
                    print(f"[{time_}] | ❌ BUY Stoploss hit @ {sl_price:.2f} | Closed remaining {book_qty:.5f}")
                open_trade = None
                continue

            if side == "SELL" and price >= t["sl"]:
                sl_price = t["sl"]
                book_qty = t["remaining_qty"]
                if book_qty > 0:
                    pnl = (t["entry_price"] - sl_price) * book_qty
                    trades.append({**t, "exit_price": sl_price, "exit_time": time_,
                                   "reason": "SL", "pnl": pnl, "qty": book_qty})
                    print(f"[{time_}] | ❌ SELL Stoploss hit @ {sl_price:.2f} | Closed remaining {book_qty:.5f}")
                open_trade = None
                continue

            # 🎯 TARGET BOOKINGS
            for idx in range(5):
                if t["booked"][idx] == 0:
                    if side == "BUY" and price >= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["targets"][idx] - t["entry_price"]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx],
                                           "exit_time": time_, "reason": f"TP{idx+1}",
                                           "pnl": pnl, "qty": book_qty})
                            print(f"[{time_}] | 🎯 BUY TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")

                    elif side == "SELL" and price <= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["entry_price"] - t["targets"][idx]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx],
                                           "exit_time": time_, "reason": f"TP{idx+1}",
                                           "pnl": pnl, "qty": book_qty})
                            print(f"[{time_}] | 🎯 SELL TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")

            # ⏰ EOD Close (exactly 05:00 IST)
            if time_.time() == dt.time(CONFIG["session_reset_hour"], 0) and t["remaining_qty"] > 0:
                book_qty = t["remaining_qty"]
                sl_price = t["sl"]
                pnl = (sl_price - t["entry_price"]) * book_qty if side == "BUY" else (t["entry_price"] - sl_price) * book_qty
                trades.append({**t, "exit_price": sl_price, "exit_time": time_,
                               "reason": "EOD", "pnl": pnl, "qty": book_qty})
                print(f"[{time_}] | ⏰ EOD Close | {side} @ {sl_price:.2f} | Remaining {book_qty:.5f}")
                open_trade = None

    return trades, open_trade

# ============================================================
# 📋 PRINT CLOSED TRADES SUMMARY
# ============================================================

def print_closed_trades(trades):
    if len(trades) == 0:
        print("No closed trades yet.\n")
        return

    print("\n📜 Closed trades (today):")
    headers = ["side", "entry_price", "entry_time", "exit_price", "exit_time", "reason", "pnl", "qty"]
    print(" | ".join([f"{h:>12}" for h in headers]))
    for t in trades:
        entry_str = t["entry_time"].strftime('%Y-%m-%d %H:%M:%S')
        exit_str = t["exit_time"].strftime('%Y-%m-%d %H:%M:%S')
        print(" | ".join([
            f"{t['side']:>6}",
            f"{t['entry_price']:>10.2f}",
            f"{entry_str:>19}",
            f"{t['exit_price']:>10.2f}",
            f"{exit_str:>19}",
            f"{t['reason']:>8}",
            f"{t['pnl']:>8.2f}",
            f"{t['qty']:>8.5f}"
        ]))

# ============================================================
# ▶️ MAIN STRATEGY EXECUTION
# ============================================================

def run_orb_strategy():
    df = fetch_intraday_data(CONFIG["symbol"], CONFIG["interval"], days=2)
    session_start = get_session_start()
    df_today = df[df.index >= session_start]

    or_high, or_low, or_mid = get_opening_range(df_today, session_start)

    # Apply leverage 💥
    leveraged_value = CONFIG["trade_value"] * CONFIG["leverage"]
    trade_qty = round(leveraged_value / df_today["Close"].iloc[-1], 5)

    print(f"\n📊 Opening Range:")
    print(f"  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
    print(f"  Using Leverage: {CONFIG['leverage']}x | Effective Position Value: {leveraged_value} USDT\n")

    trades_today, open_trade = reconstruct_trades_after_or(df_today, or_high, or_low, or_mid, trade_qty)

    total_pnl = sum(t["pnl"] for t in trades_today)
    print_closed_trades(trades_today)
    print(f"\n💰 Closed PnL: {total_pnl:.2f} USDT\n")

    if open_trade:
        print("🔔 Open trade restored:")
        print(f"  Side: {open_trade['side']} | Entry: {open_trade['entry_price']:.2f} | SL: {open_trade['sl']:.2f}")
        print(f"  Remaining Qty: {open_trade['remaining_qty']:.5f}")
        print("  Targets:")
        for j, t in enumerate(open_trade["targets"], start=1):
            print(f"   → TP{j}: {t:.2f}")
    else:
        print("No open trades currently.\n")

# ============================================================
# 🚀 RUN
# ============================================================

if __name__ == "__main__":
    run_orb_strategy()



📊 Opening Range:
  OR High 108668.37 | Low 108275.28 | Mid 108471.82
  Using Leverage: 20x | Effective Position Value: 4000 USDT

[2025-10-31 06:15:00+05:30] | 📈 BUY triggered @ 108795.49 | SL 108471.82
🎯 Targets:
   → TP1: 109188.58
   → TP2: 109581.67
   → TP3: 109974.76
   → TP4: 110367.85
   → TP5: 110760.94
[2025-10-31 06:25:00+05:30] | 🎯 BUY TP1 hit @ 109188.58 | Remaining 0.03094
[2025-10-31 06:40:00+05:30] | 🎯 BUY TP2 hit @ 109581.67 | Remaining 0.02548
[2025-10-31 06:40:00+05:30] | 🎯 BUY TP3 hit @ 109974.76 | Remaining 0.01820

📜 Closed trades (today):
        side |  entry_price |   entry_time |   exit_price |    exit_time |       reason |          pnl |          qty
   BUY |  108795.49 | 2025-10-31 06:15:00 |  109188.58 | 2025-10-31 06:25:00 |      TP1 |     2.15 |  0.00546
   BUY |  108795.49 | 2025-10-31 06:15:00 |  109581.67 | 2025-10-31 06:40:00 |      TP2 |     4.29 |  0.00546
   BUY |  108795.49 | 2025-10-31 06:15:00 |  109974.76 | 2025-10-31 06:40:00 |      TP3 |    

In [64]:
# ============================================================
# FINAL: ORB Live (mainnet data) + Testnet Futures orders (6-part brackets)
# - Keeps your working simulated reconstruct logic unchanged
# - Uses testnet for order placement
# - 6 parts: first 5 with TP+SL, 6th only SL (runner)
# - Rounds prices & qty to exchange tick/step sizes to avoid precision errors
# ============================================================

import os
import time
import pandas as pd
import datetime as dt
import pytz
from binance.client import Client
from binance.enums import *

# ----------------------------
# === CONFIG (change keys if needed) ===
# ----------------------------
LIVE_API_KEY = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
LIVE_API_SECRET = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"

TEST_API_KEY = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
TEST_API_SECRET = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

SYMBOL = "BTCUSDT"
CANDLE_INTERVAL = "5m"            # you asked for 5m entry checks
OPENING_RANGE_MINS = 30           # 5:30 - 6:00 ORB
TRADE_VALUE_USDT = 200            # capital in USDT
LEVERAGE = 20
SESSION_RESET_HOUR = 5            # reset at 05:00 IST as you wanted
TZ = pytz.timezone("Asia/Kolkata")

# ----------------------------
# === Clients ===
# ----------------------------
live_client = Client(LIVE_API_KEY, LIVE_API_SECRET)           # for market data
test_client = Client(TEST_API_KEY, TEST_API_SECRET, testnet=True)  # for testnet orders

# ----------------------------
# === Exchange info (tick/step sizes) helper ===
# ----------------------------
def get_symbol_filters(client, symbol):
    info = client.futures_exchange_info()
    s = next((x for x in info["symbols"] if x["symbol"] == symbol), None)
    if s is None:
        raise RuntimeError("Symbol not found in exchange info")
    price_filter = next(f for f in s["filters"] if f["filterType"] == "PRICE_FILTER")
    lot_size = next(f for f in s["filters"] if f["filterType"] == "LOT_SIZE")
    tick = float(price_filter["tickSize"])
    step = float(lot_size["stepSize"])
    # compute precisions for convenience
    def precision_from_step(x):
        p=0
        s=str(x)
        if "e-" in s:
            # scientific
            p = abs(int(s.split("e-")[1]))
        elif "." in s:
            p = len(s.split(".")[1].rstrip('0'))
        return p
    return {"tick": tick, "step": step, "price_prec": precision_from_step(tick), "qty_prec": precision_from_step(step)}

symbol_filters = get_symbol_filters(test_client, SYMBOL)

def round_price_to_tick(p):
    # round to nearest tick using tick size
    tick = symbol_filters["tick"]
    prec = symbol_filters["price_prec"]
    rounded = round(round(p / tick) * tick, prec)
    return rounded

def round_qty_step(q):
    step = symbol_filters["step"]
    prec = symbol_filters["qty_prec"]
    rounded = round(round(q / step) * step, prec)
    # ensure non-zero positive
    if rounded <= 0:
        rounded = round(step, prec)
    return rounded

# ----------------------------
# === Data fetcher ===
# ----------------------------
def fetch_intraday_data(symbol, interval, days=2):
    # use live_client (mainnet) for candles
    klines = live_client.get_historical_klines(symbol, interval, f"{days} day ago UTC")
    df = pd.DataFrame(klines, columns=[
        "Open time","Open","High","Low","Close","Volume",
        "Close time","Quote asset volume","Number of trades",
        "Taker buy base","Taker buy quote","Ignore"
    ])
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time").tz_convert(TZ)
    df = df[["Open","High","Low","Close","Volume"]].astype(float)
    return df

# ----------------------------
# === Session & ORB helpers ===
# ----------------------------
def get_session_start(now=None):
    now = now or dt.datetime.now(TZ)
    # session start is 05:30 that day
    session_start = now.replace(hour=5, minute=30, second=0, microsecond=0)
    if now.time() < dt.time(5, 30):
        session_start -= dt.timedelta(days=1)
    return session_start

def get_opening_range(df, session_start):
    start = session_start
    end = start + dt.timedelta(minutes=OPENING_RANGE_MINS)
    or_df = df[(df.index >= start) & (df.index < end)]
    if or_df.empty:
        raise ValueError("Opening range window empty.")
    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2
    return or_high, or_low, or_mid

# ----------------------------
# === Reconstruct trades (identical logic you had) ===
# returns trades_list (closed), open_trade (dict or None)
# ----------------------------
def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None

    range_size = or_high - or_low
    X = range_size / 2

    fractions = [0.25, 0.25, 0.15, 0.15, 0.10, 0.10]
    multipliers = [2, 4, 6, 8, 10]  # T1-T5

    for i in range(1, len(df)):
        price = df["Close"].iloc[i]
        time_ = df.index[i]

        # consider only completed 5-minute candles for entries and breakout checks
        if time_.minute % 5 != 0:
            continue

        # ENTRY when not in trade
        if open_trade is None:
            if price > or_high:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "BUY", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6
                }
                # targets computed as entry + (X * multipliers)
                open_trade["targets"] = [entry + (X*m) for m in multipliers]
                print(f"[{time_}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j,tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

            if price < or_low:
                entry = price
                sl = or_mid
                open_trade = {
                    "side": "SELL", "entry_price": entry, "entry_time": time_,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6
                }
                open_trade["targets"] = [entry - (X*m) for m in multipliers]
                print(f"[{time_}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j,tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

        # IF in trade => check SL & TP bookings
        if open_trade:
            t = open_trade
            side = t["side"]
            # STOPLOSS (use OR mid as exact hit price)
            if side == "BUY" and price <= t["sl"]:
                pnl = (t["sl"] - t["entry_price"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_, "reason": "SL", "pnl": pnl})
                print(f"[{time_}] | ❌ BUY Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue
            if side == "SELL" and price >= t["sl"]:
                pnl = (t["entry_price"] - t["sl"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_, "reason": "SL", "pnl": pnl})
                print(f"[{time_}] | ❌ SELL Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue

            # TARGET bookings (T1-T5)
            for idx in range(5):
                if t["booked"][idx] == 0:
                    if side == "BUY" and price >= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["targets"][idx] - t["entry_price"]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_,
                                           "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{time_}] | 🎯 BUY TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")

                    elif side == "SELL" and price <= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["entry_price"] - t["targets"][idx]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_,
                                           "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{time_}] | 🎯 SELL TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")

    return trades, open_trade

# ----------------------------
# === Testnet order placer: 6 parts, rounds price & qty properly ===
# ----------------------------
def place_testnet_6part(side, entry_price, or_mid, or_range, trade_qty):
    """
    Places 6 entries on testnet:
      - parts 1..5: entry (market) + TP (limit) + SL (stop-market)
      - part 6: entry (market) + SL only (no TP)
    All prices/qty rounded per exchange filters to avoid precision errors.
    """
    print("📦 Placing testnet 6-part bracket orders...")

    # split: 15,15,20,25,15,10 (your earlier preferred fractions)
    #fractions = [0.15, 0.15, 0.20, 0.25, 0.15, 0.10]
    fractions = [0.25, 0.25, 0.15, 0.15, 0.10, 0.10]
    # multipliers for TPs as earlier: X,2X,3X,4X,5X etc. In reconstruct we used X=range/2 and multipliers [2,4,6,8,10]
    # Here or_range should be X (range/2) or full? To match your earlier outputs where TP1 = entry + (X*2) etc,
    # we'll compute as in reconstruct: X = or_range, TP_i = entry +/- (X * multipliers[i]).
    multipliers = [2,4,6,8,10]  # T1..T5

    for i in range(6):
        # compute qty for this part (round to step)
        sub_qty = round_qty_step(trade_qty * fractions[i])
        if sub_qty <= 0:
            print(f"⚠️ sub_qty for part {i+1} evaluated to zero; skipping")
            continue

        # Entry market order
        try:
            if side == "BUY":
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_BUY, type=ORDER_TYPE_MARKET, quantity=sub_qty)
            else:
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_SELL, type=ORDER_TYPE_MARKET, quantity=sub_qty)
        except Exception as e:
            print(f"⚠️ Error placing market entry for part {i+1}: {e}")
            # continue attempt next parts (do not abort whole function)

        # compute TP price for parts 1..5 (part 6 has no TP)
        if i < 5:
            if side == "BUY":
                tp_raw = entry_price + (or_range * multipliers[i])
            else:
                tp_raw = entry_price - (or_range * multipliers[i])
            tp_price = round_price_to_tick(tp_raw)
        else:
            tp_price = None

        # SL is OR mid for all parts
        sl_price = round_price_to_tick(or_mid)

        # Place TP limit (reduceOnly) for parts 1..5
        if tp_price is not None:
            try:
                if side == "BUY":
                    test_client.futures_create_order(
                        symbol=SYMBOL,
                        side=SIDE_SELL,
                        type=ORDER_TYPE_LIMIT,
                        timeInForce=TIME_IN_FORCE_GTC,
                        price=tp_price,
                        quantity=sub_qty,
                        reduceOnly=True
                    )
                else:
                    test_client.futures_create_order(
                        symbol=SYMBOL,
                        side=SIDE_BUY,
                        type=ORDER_TYPE_LIMIT,
                        timeInForce=TIME_IN_FORCE_GTC,
                        price=tp_price,
                        quantity=sub_qty,
                        reduceOnly=True
                    )
            except Exception as e:
                print(f"⚠️ Error placing TP (part {i+1}): {e}")

        # Place SL stop-market (reduceOnly) for each part
        try:
            if side == "BUY":
                test_client.futures_create_order(
                    symbol=SYMBOL,
                    side=SIDE_SELL,
                    type="STOP_MARKET",
                    stopPrice=sl_price,
                    closePosition=False,
                    quantity=sub_qty,
                    reduceOnly=True
                )
            else:
                test_client.futures_create_order(
                    symbol=SYMBOL,
                    side=SIDE_BUY,
                    type="STOP_MARKET",
                    stopPrice=sl_price,
                    closePosition=False,
                    quantity=sub_qty,
                    reduceOnly=True
                )
        except Exception as e:
            print(f"⚠️ Error placing SL (part {i+1}): {e}")

        # brief pause to avoid rate limits
        time.sleep(0.12)

    print("🎯 Testnet 6-part orders submitted.\n")

# ----------------------------
# === MAIN: reconstruct history then go live loop (no duplicate entries) ===
# ----------------------------
def run_orb_live():
    print("🚀 Starting ORB Live Simulation + Testnet (mainnet data + testnet orders)\n")

    # fetch recent candles (2 days to be safe)
    df = fetch_intraday_data(SYMBOL, CANDLE_INTERVAL, days=2)

    session_start = get_session_start()
    df_today = df[df.index >= session_start]

    # compute/opening range
    try:
        or_high, or_low, or_mid = get_opening_range(df_today, session_start)
    except Exception as e:
        print("⚠️ Opening range error:", e)
        return

    # trade quantity in base asset considering leverage
    last_price = df_today["Close"].iloc[-1]
    leveraged_value = TRADE_VALUE_USDT * LEVERAGE
    trade_qty = round_qty_step(leveraged_value / last_price)  # qty in BTC (rounded)

    # Print ORB at top
    print(f"\n📊 Opening Range (reconstructed):\n  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}\n")

    # Reconstruct today's trades and restore open trade if present
    trades_today, open_trade = reconstruct_trades_after_or(df_today, or_high, or_low, or_mid, trade_qty)

    # Print closed trades summary
    if trades_today:
        # drop duplicates and show concise table
        closed_df = pd.DataFrame(trades_today)
        # ensure columns exist
        cols = ["side","entry_price","entry_time","exit_price","exit_time","reason","pnl","qty"]
        present_cols = [c for c in cols if c in closed_df.columns]
        print("📜 Closed trades found (today):")
        print(closed_df[present_cols].to_string(index=False))
    else:
        print("📜 Closed trades found (today): None")

    closed_pnl = sum(t.get("pnl",0) for t in trades_today)

    # If an open trade restored from history, do not place new entry until it closes
    if open_trade:
        # compute open pnl using last price
        last_price = df_today["Close"].iloc[-1]
        if open_trade["side"] == "BUY":
            open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
        else:
            open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]
        total_pnl = closed_pnl + open_pnl
        print(f"\n🔔 Open trade restored:")
        print(f"  Side: {open_trade['side']} | Entry: {open_trade['entry_price']:.2f} | SL: {open_trade['sl']:.2f}")
        print(f"  Remaining Qty: {open_trade['remaining_qty']:.5f}")
        print(f"\n💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {total_pnl:.2f}\n")
    else:
        print(f"\n💰 Closed PnL: {closed_pnl:.2f} | Open PnL: 0.00 | Total: {closed_pnl:.2f}\n")

    # ----------------------------
    # Live monitoring loop (checks every minute; places orders only if open_trade is None)
    # ----------------------------
    testnet_orders_placed = False
    while True:
        try:
            now = dt.datetime.now(TZ)
            df = fetch_intraday_data(SYMBOL, CANDLE_INTERVAL, days=2)
            df_today = df[df.index >= session_start]
            last_price = df_today["Close"].iloc[-1]
            print(f"[{now.strftime('%Y-%m-%d %H:%M:%S %Z')}] Current price: {last_price:.2f}")

            # recompute ORB if needed (same session)
            # (we keep the same or_high/or_low/or_mid for the session)
            # skip new entry if we have an open_trade restored earlier or that was created live
            if open_trade:
                # compute open pnl
                if open_trade["side"] == "BUY":
                    open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
                else:
                    open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]
                closed_pnl = sum(t.get("pnl",0) for t in trades_today)
                total_pnl = closed_pnl + open_pnl
                print(f"💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {total_pnl:.2f}")
                # check EOD reset: if now >= next reset (05:00) then close open trade logically
                reset_dt = session_start.replace(hour=SESSION_RESET_HOUR, minute=0, second=0, microsecond=0)
                if now.time() >= dt.time(SESSION_RESET_HOUR,0) and now.date() != session_start.date():
                    print("🕔 EOD Reset reached — clearing open_trade for new session.")
                    open_trade = None
                    testnet_orders_placed = False
                time.sleep(60)
                continue

            # No open trades -> check for new entry (only at completed 5-min candles)
            # ensure we check only at minute %5 ==0 to match your rule (but we still run loop every minute)
            if last_price is None:
                time.sleep(10)
                continue

            # check breakout on completed 5-min candle timestamps
            last_idx = df_today.index[-1]
            # make sure last index corresponds to closed 5-min candle
            if last_idx.minute % 5 != 0:
                # wait until next completed 5-min candle
                time.sleep(10)
                continue

            # fresh entry condition
            if last_price > or_high and not testnet_orders_placed:
                print(f"[{last_idx}] | 📈 TRUE BREAKOUT BUY @ {last_price:.2f}")
                # place 6-part testnet orders
                # or_range for TP calc should match reconstruct (use X = (or_high - or_low)/2)
                X = (or_high - or_low) / 2
                # use trade_qty computed earlier (trade_qty variable)
                # we recompute trade_qty in case price moved a lot
                leveraged_value = TRADE_VALUE_USDT * LEVERAGE
                trade_qty_live = round_qty_step(leveraged_value / last_price)
                # place orders only if no open_trade
                place_testnet_6part("BUY", last_price, or_mid, X, trade_qty_live)
                # create an in-memory open_trade to avoid duplicate entries until it's closed
                open_trade = {"side":"BUY","entry_price":last_price,"entry_time":last_idx,"sl":or_mid,"qty":trade_qty_live,"remaining_qty":trade_qty_live}
                testnet_orders_placed = True

            elif last_price < or_low and not testnet_orders_placed:
                print(f"[{last_idx}] | 📉 TRUE BREAKOUT SELL @ {last_price:.2f}")
                X = (or_high - or_low) / 2
                leveraged_value = TRADE_VALUE_USDT * LEVERAGE
                trade_qty_live = round_qty_step(leveraged_value / last_price)
                place_testnet_6part("SELL", last_price, or_mid, X, trade_qty_live)
                open_trade = {"side":"SELL","entry_price":last_price,"entry_time":last_idx,"sl":or_mid,"qty":trade_qty_live,"remaining_qty":trade_qty_live}
                testnet_orders_placed = True

        except Exception as e:
            print("⚠️ Runtime error in live loop:", e)

        # sleep 30-60 seconds to reduce API calls
        time.sleep(30)

# ----------------------------
if __name__ == "__main__":
    run_orb_live()


🚀 Starting ORB Live Simulation + Testnet (mainnet data + testnet orders)


📊 Opening Range (reconstructed):
  OR High 110127.26 | Low 109895.27 | Mid 110011.26

[2025-11-02 07:45:00+05:30] | 📉 SELL triggered @ 109893.94 | SL 110011.26
🎯 Targets:
   → TP1: 109661.95
   → TP2: 109429.96
   → TP3: 109197.97
   → TP4: 108965.98
   → TP5: 108733.99
[2025-11-02 08:00:00+05:30] | ❌ SELL Stoploss hit @ 110011.26
[2025-11-02 09:50:00+05:30] | 📈 BUY triggered @ 110129.39 | SL 110011.26
🎯 Targets:
   → TP1: 110361.38
   → TP2: 110593.37
   → TP3: 110825.36
   → TP4: 111057.35
   → TP5: 111289.34
[2025-11-02 10:10:00+05:30] | 🎯 BUY TP1 hit @ 110361.38 | Remaining 0.02700
[2025-11-02 10:25:00+05:30] | 🎯 BUY TP2 hit @ 110593.37 | Remaining 0.01800
[2025-11-02 13:20:00+05:30] | 🎯 BUY TP3 hit @ 110825.36 | Remaining 0.01260
[2025-11-02 17:20:00+05:30] | 🎯 BUY TP4 hit @ 111057.35 | Remaining 0.00720
[2025-11-02 21:30:00+05:30] | ❌ BUY Stoploss hit @ 110011.26
[2025-11-02 21:35:00+05:30] | 📉 SELL trigge

KeyboardInterrupt: 

In [ ]:
#testnet
api = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
secret = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

In [ ]:
api = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
secret = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

In [63]:
import pandas as pd
from datetime import time
from binance.client import Client

# === Connect to Binance Futures ===
API_KEY = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
API_SECRET = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"

client = Client(api_key=API_KEY, api_secret=API_SECRET, testnet=False)

symbol = "BTCUSDT"
interval = "5m"

# ✅ Fetch Binance Futures Perpetual data (always in UTC)
klines = client.futures_klines(symbol=symbol, interval=interval, limit=200)
df = pd.DataFrame(klines, columns=[
    "timestamp", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "trades", "taker_buy_base",
    "taker_buy_quote", "ignore"
])

# Convert timestamps to UTC datetimes
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
df.set_index("timestamp", inplace=True)
df = df.astype(float)

# === ORB Calculation ===
# 5:30AM–6:00AM IST = 00:00–00:30 UTC (Binance server time)
or_start = time(0, 0)
or_end = time(0, 30)

# Select candles within ORB window (UTC)
or_data = df.between_time(or_start, or_end, inclusive="left")

or_high = or_data["high"].max()
or_low = or_data["low"].min()
or_mid = (or_high + or_low) / 2
or_range = or_high - or_low

# Round to BTCUSDT tick size (0.1)
tick_size = 0.1
or_high = round(or_high / tick_size) * tick_size
or_low = round(or_low / tick_size) * tick_size
or_mid = round(or_mid / tick_size) * tick_size
or_range = round(or_range, 2)

# Convert to IST for print/logs
ist_or_data = or_data.tz_convert("Asia/Kolkata")
ist_start = ist_or_data.index[0].strftime("%H:%M:%S %Z") if not ist_or_data.empty else "N/A"
ist_end = ist_or_data.index[-1].strftime("%H:%M:%S %Z") if not ist_or_data.empty else "N/A"

# ✅ Print output
print("📊 Opening Range (Binance UTC | Printed in IST):")
print(f"  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
print(f"  Range: {or_range:.2f} | Time Window: {ist_start} → {ist_end}")


📊 Opening Range (Binance UTC | Printed in IST):
  OR High 110088.00 | Low 109846.70 | Mid 109967.40
  Range: 241.30 | Time Window: 05:30:00 IST → 05:55:00 IST


In [44]:
# ============================================================
# FINAL: ORB Live (mainnet futures data) + Testnet Futures orders (6-part brackets)
# - Keeps your working simulated reconstruct logic unchanged
# - Uses testnet for order placement
# - 6 parts: first 5 with TP+SL, 6th only SL (runner)
# - Rounds prices & qty to exchange tick/step sizes to avoid precision errors
# - ORB window: 05:30-06:00 IST => 00:00-00:30 UTC (Binance)
# - Logic runs in UTC, prints/logs in IST
# ============================================================

import os
import time
import pandas as pd
import datetime as dt
import pytz
from binance.client import Client
from binance.enums import *

# ----------------------------
# === CONFIG (change keys if needed) ===
# ----------------------------
LIVE_API_KEY = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
LIVE_API_SECRET = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"

TEST_API_KEY = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
TEST_API_SECRET = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

SYMBOL = "BTCUSDT"
CANDLE_INTERVAL = "5m"            # 5m entry checks
OPENING_RANGE_MINS = 30           # 5:30 - 6:00 IST => 00:00 - 00:30 UTC
TRADE_VALUE_USDT = 200            # capital in USDT
LEVERAGE = 20
SESSION_RESET_HOUR = 5            # reset at 05:00 IST
TZ = pytz.timezone("Asia/Kolkata")

# ----------------------------
# === Clients ===
# ----------------------------
live_client = Client(LIVE_API_KEY, LIVE_API_SECRET)                 # for market data (futures)
test_client = Client(TEST_API_KEY, TEST_API_SECRET, testnet=True)  # for testnet orders

# ----------------------------
# === Exchange info (tick/step sizes) helper ===
# ----------------------------
def get_symbol_filters(client, symbol):
    info = client.futures_exchange_info()
    s = next((x for x in info["symbols"] if x["symbol"] == symbol), None)
    if s is None:
        raise RuntimeError("Symbol not found in exchange info")
    price_filter = next(f for f in s["filters"] if f["filterType"] == "PRICE_FILTER")
    lot_size = next(f for f in s["filters"] if f["filterType"] == "LOT_SIZE")
    tick = float(price_filter["tickSize"])
    step = float(lot_size["stepSize"])
    # compute precisions for convenience
    def precision_from_step(x):
        p=0
        s=str(x)
        if "e-" in s:
            p = abs(int(s.split("e-")[1]))
        elif "." in s:
            p = len(s.split(".")[1].rstrip('0'))
        return p
    return {"tick": tick, "step": step, "price_prec": precision_from_step(tick), "qty_prec": precision_from_step(step)}

symbol_filters = get_symbol_filters(test_client, SYMBOL)

def round_price_to_tick(p):
    tick = symbol_filters["tick"]
    prec = symbol_filters["price_prec"]
    rounded = round(round(p / tick) * tick, prec)
    return rounded

def round_qty_step(q):
    step = symbol_filters["step"]
    prec = symbol_filters["qty_prec"]
    rounded = round(round(q / step) * step, prec)
    if rounded <= 0:
        rounded = round(step, prec)
    return rounded

# ----------------------------
# === Helpers for time display ===
# ----------------------------
def to_ist(ts):
    """ Convert a tz-aware UTC Timestamp to IST string """
    try:
        return ts.tz_convert(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    except Exception:
        # if ts is naive or not tz-aware, attempt to localize then convert
        try:
            ts_utc = pd.to_datetime(ts).tz_localize(dt.timezone.utc)
            return ts_utc.tz_convert(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
        except Exception:
            return str(ts)

# ----------------------------
# === Data fetcher (Binance Futures, keep index in UTC) ===
# ----------------------------
def fetch_intraday_data(symbol, interval, days=2):
    # Use futures_klines to match Binance Perpetual
    klines = live_client.futures_klines(symbol=symbol, interval=interval, limit= (int(days)*24*60)//(int(interval.replace('m',''))))
    df = pd.DataFrame(klines, columns=[
        "Open time","Open","High","Low","Close","Volume",
        "Close time","Quote asset volume","Number of trades",
        "Taker buy base","Taker buy quote","Ignore"
    ])
    # Binance returns ms epoch UTC - keep as UTC index
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time")  # tz-aware UTC index
    df = df[["Open","High","Low","Close","Volume"]].astype(float)
    return df

# ----------------------------
# === Session & ORB helpers (WORK IN UTC) ===
# ----------------------------
def get_session_start(now_utc=None):
    """
    Session start for ORB logic in UTC:
    - 5:30 IST = 00:00 UTC
    We return session_start_utc as midnight UTC of the session day.
    """
    now_utc = now_utc or dt.datetime.now(dt.timezone.utc)
    # session_start_utc = same date at 00:00 UTC
    session_start_utc = now_utc.replace(hour=0, minute=0, second=0, microsecond=0)
    # If current UTC time is before 00:00 (never), subtract a day; safe guard:
    if now_utc.time() < dt.time(0,0):
        session_start_utc -= dt.timedelta(days=1)
    return session_start_utc

def get_opening_range(df, session_start_utc):
    """
    OR window: 00:00 - 00:30 UTC (corresponds to 05:30 - 06:00 IST)
    df index is expected to be tz-aware UTC.
    """
    start = session_start_utc  # 00:00 UTC
    end = start + dt.timedelta(minutes=OPENING_RANGE_MINS)  # +30 -> 00:30 UTC
    or_df = df[(df.index >= start) & (df.index < end)]
    if or_df.empty:
        raise ValueError("Opening range window empty.")
    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2
    # Round to tick to be consistent with exchange
    or_high = round_price_to_tick(or_high)
    or_low = round_price_to_tick(or_low)
    or_mid = round_price_to_tick(or_mid)
    return or_high, or_low, or_mid

# ----------------------------
# === Reconstruct trades (unchanged logic except time printing uses IST helper) ===
# returns trades_list (closed), open_trade (dict or None)
# ----------------------------
def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None

    range_size = or_high - or_low
    X = range_size / 2

    # fractions you set earlier in the code block
    fractions = [0.15, 0.20, 0.25, 0.20, 0.10, 0.10]
    multipliers = [2, 4, 6, 8, 10]  # T1-T5

    for i in range(1, len(df)):
        close = df["Close"].iloc[i]
        high = df["High"].iloc[i]
        low = df["Low"].iloc[i]
        time_utc = df.index[i]

        if time_utc.minute % 5 != 0:
            continue

        # ENTRY when not in trade
        if open_trade is None:
            if close > or_high:
                entry = close
                sl = or_mid
                open_trade = {
                    "side": "BUY", "entry_price": entry, "entry_time": time_utc,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6
                }
                open_trade["targets"] = [entry + (X*m) for m in multipliers]
                print(f"[{to_ist(time_utc)}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j,tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

            if close < or_low:
                entry = close
                sl = or_mid
                open_trade = {
                    "side": "SELL", "entry_price": entry, "entry_time": time_utc,
                    "sl": sl, "qty": trade_qty,
                    "remaining_qty": trade_qty,
                    "booked": [0]*6
                }
                open_trade["targets"] = [entry - (X*m) for m in multipliers]
                print(f"[{to_ist(time_utc)}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j,tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

        # IF in trade => check SL & TP bookings
        if open_trade:
            t = open_trade
            side = t["side"]

            # STOPLOSS
            if side == "BUY" and low <= t["sl"]:
                pnl = (t["sl"] - t["entry_price"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_utc, "reason": "SL", "pnl": pnl})
                print(f"[{to_ist(time_utc)}] | ❌ BUY Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue

            if side == "SELL" and high >= t["sl"]:
                pnl = (t["entry_price"] - t["sl"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_utc, "reason": "SL", "pnl": pnl})
                print(f"[{to_ist(time_utc)}] | ❌ SELL Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue

            # TARGET bookings (T1–T5)
            for idx in range(5):
                if t["booked"][idx] == 0:
                    if side == "BUY" and high >= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["targets"][idx] - t["entry_price"]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_utc,
                                           "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{to_ist(time_utc)}] | 🎯 BUY TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")

                    elif side == "SELL" and low <= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["entry_price"] - t["targets"][idx]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_utc,
                                           "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{to_ist(time_utc)}] | 🎯 SELL TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")

    return trades, open_trade

# ----------------------------
# === Testnet order placer: 6 parts, rounds price & qty properly ===
# ----------------------------
def place_testnet_6part(side, entry_price, or_mid, or_range, trade_qty):
    print("📦 Placing testnet 6-part bracket orders...")
    fractions = [0.15, 0.20, 0.25, 0.20, 0.10, 0.10]
    #fractions = [0.25, 0.25, 0.15, 0.15, 0.10, 0.10]
    multipliers = [2,4,6,8,10]  # T1..T5

    for i in range(6):
        sub_qty = round_qty_step(trade_qty * fractions[i])
        if sub_qty <= 0:
            print(f"⚠️ sub_qty for part {i+1} evaluated to zero; skipping")
            continue

        # Market entry
        try:
            if side == "BUY":
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_BUY, type=ORDER_TYPE_MARKET, quantity=sub_qty)
            else:
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_SELL, type=ORDER_TYPE_MARKET, quantity=sub_qty)
        except Exception as e:
            print(f"⚠️ Error placing market entry for part {i+1}: {e}")

        # TP price for parts 1..5
        if i < 5:
            if side == "BUY":
                tp_raw = entry_price + (or_range * multipliers[i])
            else:
                tp_raw = entry_price - (or_range * multipliers[i])
            tp_price = round_price_to_tick(tp_raw)
        else:
            tp_price = None

        sl_price = round_price_to_tick(or_mid)

        # Place TP limit (reduceOnly) for parts 1..5
        if tp_price is not None:
            try:
                if side == "BUY":
                    test_client.futures_create_order(
                        symbol=SYMBOL,
                        side=SIDE_SELL,
                        type=ORDER_TYPE_LIMIT,
                        timeInForce=TIME_IN_FORCE_GTC,
                        price=tp_price,
                        quantity=sub_qty,
                        reduceOnly=True
                    )
                else:
                    test_client.futures_create_order(
                        symbol=SYMBOL,
                        side=SIDE_BUY,
                        type=ORDER_TYPE_LIMIT,
                        timeInForce=TIME_IN_FORCE_GTC,
                        price=tp_price,
                        quantity=sub_qty,
                        reduceOnly=True
                    )
            except Exception as e:
                print(f"⚠️ Error placing TP (part {i+1}): {e}")

        # Place SL stop-market (reduceOnly) for each part
        try:
            if side == "BUY":
                test_client.futures_create_order(
                    symbol=SYMBOL,
                    side=SIDE_SELL,
                    type="STOP_MARKET",
                    stopPrice=sl_price,
                    closePosition=False,
                    quantity=sub_qty,
                    reduceOnly=True
                )
            else:
                test_client.futures_create_order(
                    symbol=SYMBOL,
                    side=SIDE_BUY,
                    type="STOP_MARKET",
                    stopPrice=sl_price,
                    closePosition=False,
                    quantity=sub_qty,
                    reduceOnly=True
                )
        except Exception as e:
            print(f"⚠️ Error placing SL (part {i+1}): {e}")

        time.sleep(0.12)

    print("🎯 Testnet 6-part orders submitted.\n")

# ----------------------------
# === MAIN: reconstruct history then go live loop (no duplicate entries) ===
# ----------------------------
def run_orb_live():
    print("🚀 Starting ORB Live Simulation + Testnet (mainnet futures data + testnet orders)\n")

    # fetch recent candles (2 days)
    df = fetch_intraday_data(SYMBOL, CANDLE_INTERVAL, days=2)

    # session_start in UTC (00:00 UTC corresponds to 05:30 IST)
    session_start_utc = get_session_start()
    df_today = df[df.index >= session_start_utc]

    # compute/opening range (uses UTC window 00:00-00:30)
    try:
        or_high, or_low, or_mid = get_opening_range(df_today, session_start_utc)
    except Exception as e:
        print("⚠️ Opening range error:", e)
        return

    last_price = df_today["Close"].iloc[-1]
    leveraged_value = TRADE_VALUE_USDT * LEVERAGE
    trade_qty = round_qty_step(leveraged_value / last_price)

    # Print ORB at top (show IST window times)
    ist_start = (session_start_utc).astimezone(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    ist_end = (session_start_utc + dt.timedelta(minutes=OPENING_RANGE_MINS)).astimezone(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    print(f"\n📊 Opening Range (Binance UTC; printed in IST):\n  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
    print(f"  OR Window: {ist_start} → {ist_end}\n")

    # Reconstruct today's trades and restore open trade if present
    trades_today, open_trade = reconstruct_trades_after_or(df_today, or_high, or_low, or_mid, trade_qty)

    # Print closed trades summary
    if trades_today:
        closed_df = pd.DataFrame(trades_today)
        cols = ["side","entry_price","entry_time","exit_price","exit_time","reason","pnl","qty"]
        present_cols = [c for c in cols if c in closed_df.columns]
        # Convert UTC timestamps to IST in the dataframe for display
        if "entry_time" in closed_df.columns:
            closed_df["entry_time"] = closed_df["entry_time"].apply(lambda x: to_ist(x))
        if "exit_time" in closed_df.columns:
            closed_df["exit_time"] = closed_df["exit_time"].apply(lambda x: to_ist(x))
        print("📜 Closed trades found (today):")
        print(closed_df[present_cols].to_string(index=False))
    else:
        print("📜 Closed trades found (today): None")

    closed_pnl = sum(t.get("pnl",0) for t in trades_today)

    # If an open trade restored from history, do not place new entry until it closes
    if open_trade:
        last_price = df_today["Close"].iloc[-1]
        if open_trade["side"] == "BUY":
            open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
        else:
            open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]
        total_pnl = closed_pnl + open_pnl
        print(f"\n🔔 Open trade restored:")
        print(f"  Side: {open_trade['side']} | Entry: {open_trade['entry_price']:.2f} | SL: {open_trade['sl']:.2f}")
        print(f"  Remaining Qty: {open_trade['remaining_qty']:.5f}")
        print(f"\n💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {total_pnl:.2f}\n")
    else:
        print(f"\n💰 Closed PnL: {closed_pnl:.2f} | Open PnL: 0.00 | Total: {closed_pnl:.2f}\n")

    # ----------------------------
    # Live monitoring loop
    # ----------------------------
    testnet_orders_placed = False
    while True:
        try:
            now_ist = dt.datetime.now(TZ)
            now_utc = dt.datetime.now(dt.timezone.utc)
            df = fetch_intraday_data(SYMBOL, CANDLE_INTERVAL, days=2)
            df_today = df[df.index >= session_start_utc]
            last_price = df_today["Close"].iloc[-1]
            print(f"[{now_ist.strftime('%Y-%m-%d %H:%M:%S %Z')}] Current price: {last_price:.2f}")

            if open_trade:
                if open_trade["side"] == "BUY":
                    open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
                else:
                    open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]
                closed_pnl = sum(t.get("pnl",0) for t in trades_today)
                total_pnl = closed_pnl + open_pnl
                print(f"💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {total_pnl:.2f}")
                # EOD reset condition (IST 05:00) -> in UTC that's previous day 23:30; simplified check:
                # if date moved (new session) reset
                if now_utc.date() != session_start_utc.date():
                    print("🕔 EOD Reset reached — clearing open_trade for new session.")
                    open_trade = None
                    testnet_orders_placed = False
                time.sleep(60)
                continue

            # No open trades -> check for new entry (completed 5-min candles only)
            last_idx = df_today.index[-1]  # tz-aware UTC
            if last_idx.minute % 5 != 0:
                time.sleep(10)
                continue

            # fresh entry condition
            if last_price > or_high and not testnet_orders_placed:
                print(f"[{to_ist(last_idx)}] | 📈 TRUE BREAKOUT BUY @ {last_price:.2f}")
                X = (or_high - or_low) / 2
                leveraged_value = TRADE_VALUE_USDT * LEVERAGE
                trade_qty_live = round_qty_step(leveraged_value / last_price)
                place_testnet_6part("BUY", last_price, or_mid, X, trade_qty_live)
                open_trade = {"side":"BUY","entry_price":last_price,"entry_time":last_idx,"sl":or_mid,"qty":trade_qty_live,"remaining_qty":trade_qty_live}
                testnet_orders_placed = True

            elif last_price < or_low and not testnet_orders_placed:
                print(f"[{to_ist(last_idx)}] | 📉 TRUE BREAKOUT SELL @ {last_price:.2f}")
                X = (or_high - or_low) / 2
                leveraged_value = TRADE_VALUE_USDT * LEVERAGE
                trade_qty_live = round_qty_step(leveraged_value / last_price)
                place_testnet_6part("SELL", last_price, or_mid, X, trade_qty_live)
                open_trade = {"side":"SELL","entry_price":last_price,"entry_time":last_idx,"sl":or_mid,"qty":trade_qty_live,"remaining_qty":trade_qty_live}
                testnet_orders_placed = True

        except Exception as e:
            print("⚠️ Runtime error in live loop:", e)

        time.sleep(30)

# ----------------------------
if __name__ == "__main__":
    run_orb_live()


🚀 Starting ORB Live Simulation + Testnet (mainnet futures data + testnet orders)


📊 Opening Range (Binance UTC; printed in IST):
  OR High 109600.00 | Low 109351.50 | Mid 109475.80
  OR Window: 2025-11-01 05:30:00 IST → 2025-11-01 06:00:00 IST

[2025-11-01 06:15:00 IST] | 📈 BUY triggered @ 109694.30 | SL 109475.80
🎯 Targets:
   → TP1: 109942.80
   → TP2: 110191.30
   → TP3: 110439.80
   → TP4: 110688.30
   → TP5: 110936.80
[2025-11-01 07:00:00 IST] | 🎯 BUY TP1 hit @ 109942.80 | Remaining 0.03060
[2025-11-01 08:05:00 IST] | ❌ BUY Stoploss hit @ 109475.80
[2025-11-01 08:15:00 IST] | 📈 BUY triggered @ 109725.30 | SL 109475.80
🎯 Targets:
   → TP1: 109973.80
   → TP2: 110222.30
   → TP3: 110470.80
   → TP4: 110719.30
   → TP5: 110967.80
[2025-11-01 08:45:00 IST] | 🎯 BUY TP1 hit @ 109973.80 | Remaining 0.03060
[2025-11-01 08:45:00 IST] | 🎯 BUY TP2 hit @ 110222.30 | Remaining 0.02340
[2025-11-01 09:35:00 IST] | 🎯 BUY TP3 hit @ 110470.80 | Remaining 0.01440
📜 Closed trades found (today):
side

KeyboardInterrupt: 

In [ ]:
# ============================================================
# PRODUCTION: ORB Live (mainnet futures data) + Testnet Futures orders (6-part brackets)
# - Runs aligned to system 5-minute ticks (:00, :05, :10, ...)
# - Reconstructs trades every run, places orders only when no open trades
# - On IST 05:00 resets: cancels pending testnet orders + flattens positions
# - Uses mainnet futures for data, testnet for order placement
# ============================================================

import os
import time
import pandas as pd
import datetime as dt
import pytz
from binance.client import Client
from binance.enums import *

# ----------------------------
# === CONFIG (replace keys if needed) ===
# ----------------------------
LIVE_API_KEY = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
LIVE_API_SECRET = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"

TEST_API_KEY = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
TEST_API_SECRET = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

SYMBOL = "BTCUSDT"               # Perpetual symbol on Binance Futures
CANDLE_INTERVAL = "5m"           # entry checks on completed 5-min candles
OPENING_RANGE_MINS = 30          # ORB duration = 30 minutes (00:00-00:30 UTC -> 05:30-06:00 IST)
TRADE_VALUE_USDT = 200
LEVERAGE = 20
SESSION_RESET_HOUR = 5           # IST hour for reset (05:00 IST)
TZ = pytz.timezone("Asia/Kolkata")  # IST for printing/logs

# ----------------------------
# === Clients ===
# ----------------------------
live_client = Client(LIVE_API_KEY, LIVE_API_SECRET)                 # for market data (futures)
test_client = Client(TEST_API_KEY, TEST_API_SECRET, testnet=True)  # for testnet orders

# ----------------------------
# === Exchange info (tick/step sizes) helper ===
# ----------------------------
def get_symbol_filters(client, symbol):
    info = client.futures_exchange_info()
    s = next((x for x in info["symbols"] if x["symbol"] == symbol), None)
    if s is None:
        raise RuntimeError("Symbol not found in exchange info")
    price_filter = next(f for f in s["filters"] if f["filterType"] == "PRICE_FILTER")
    lot_size = next(f for f in s["filters"] if f["filterType"] == "LOT_SIZE")
    tick = float(price_filter["tickSize"])
    step = float(lot_size["stepSize"])
    def precision_from_step(x):
        p=0
        s=str(x)
        if "e-" in s:
            p = abs(int(s.split("e-")[1]))
        elif "." in s:
            p = len(s.split(".")[1].rstrip('0'))
        return p
    return {"tick": tick, "step": step, "price_prec": precision_from_step(tick), "qty_prec": precision_from_step(step)}

symbol_filters = get_symbol_filters(test_client, SYMBOL)

def round_price_to_tick(p):
    tick = symbol_filters["tick"]
    prec = symbol_filters["price_prec"]
    rounded = round(round(p / tick) * tick, prec)
    return rounded

def round_qty_step(q):
    step = symbol_filters["step"]
    prec = symbol_filters["qty_prec"]
    rounded = round(round(q / step) * step, prec)
    if rounded <= 0:
        rounded = round(step, prec)
    return rounded

# ----------------------------
# === Helpers for time display ===
# ----------------------------
def to_ist(ts):
    """ Convert a tz-aware UTC Timestamp to IST string """
    try:
        return ts.tz_convert(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    except Exception:
        try:
            ts_utc = pd.to_datetime(ts).tz_localize(dt.timezone.utc)
            return ts_utc.tz_convert(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
        except Exception:
            return str(ts)

# ----------------------------
# === Data fetcher (Binance Futures, keep index in UTC) ===
# ----------------------------
def fetch_intraday_data(symbol, interval, days=2):
    # use futures klines so perp/futures prices match
    minutes = int(interval.replace("m",""))
    limit = max(500, (int(days)*24*60)//minutes + 10)
    klines = live_client.futures_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        "Open time","Open","High","Low","Close","Volume",
        "Close time","Quote asset volume","Number of trades",
        "Taker buy base","Taker buy quote","Ignore"
    ])
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time")  # tz-aware UTC index
    df = df[["Open","High","Low","Close","Volume"]].astype(float)
    return df

# ----------------------------
# === Session & ORB helpers (WORK IN UTC) ===
# ----------------------------
def get_session_start(now_utc=None):
    """
    Session start for ORB logic in UTC:
    - ORB window is 00:00 - 00:30 UTC (05:30 - 06:00 IST)
    We return session_start_utc as midnight UTC of the session day.
    """
    now_utc = now_utc or dt.datetime.now(dt.timezone.utc)
    session_start_utc = now_utc.replace(hour=0, minute=0, second=0, microsecond=0)
    if now_utc.time() < dt.time(0,0):
        session_start_utc -= dt.timedelta(days=1)
    return session_start_utc

def get_opening_range(df, session_start_utc):
    start = session_start_utc  # 00:00 UTC
    end = start + dt.timedelta(minutes=OPENING_RANGE_MINS)  # +30 -> 00:30 UTC
    or_df = df[(df.index >= start) & (df.index < end)]
    if or_df.empty:
        raise ValueError("Opening range window empty.")
    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2
    or_high = round_price_to_tick(or_high)
    or_low = round_price_to_tick(or_low)
    or_mid = round_price_to_tick(or_mid)
    return or_high, or_low, or_mid

# ----------------------------
# === Reconstruct trades (kept intact, uses highs/lows for TP/SL detection) ===
# returns trades_list (closed), open_trade (dict or None)
# ----------------------------
def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None
    range_size = or_high - or_low
    X = range_size / 2
    fractions = [0.15, 0.20, 0.25, 0.20, 0.10, 0.10]   # your chosen split
    multipliers = [2, 4, 6, 8, 10]  # TP multipliers (T1..T5)

    for i in range(1, len(df)):
        close = df["Close"].iloc[i]
        high = df["High"].iloc[i]
        low = df["Low"].iloc[i]
        time_utc = df.index[i]

        # only evaluate on completed 5-min candles for breakout detection
        if time_utc.minute % 5 != 0:
            continue

        # ENTRY when not in trade
        if open_trade is None:
            if close > or_high:
                entry = close; sl = or_mid
                open_trade = {"side":"BUY","entry_price":entry,"entry_time":time_utc,"sl":sl,"qty":trade_qty,"remaining_qty":trade_qty,"booked":[0]*6}
                open_trade["targets"] = [entry + (X*m) for m in multipliers]
                print(f"[{to_ist(time_utc)}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j,tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue
            if close < or_low:
                entry = close; sl = or_mid
                open_trade = {"side":"SELL","entry_price":entry,"entry_time":time_utc,"sl":sl,"qty":trade_qty,"remaining_qty":trade_qty,"booked":[0]*6}
                open_trade["targets"] = [entry - (X*m) for m in multipliers]
                print(f"[{to_ist(time_utc)}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets:")
                for j,tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

        # IF in trade => check SL & TP bookings using high/low of candle
        if open_trade:
            t = open_trade; side = t["side"]
            # STOPLOSS: use candle low for BUY, candle high for SELL
            if side == "BUY" and low <= t["sl"]:
                pnl = (t["sl"] - t["entry_price"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_utc, "reason": "SL", "pnl": pnl})
                print(f"[{to_ist(time_utc)}] | ❌ BUY Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue
            if side == "SELL" and high >= t["sl"]:
                pnl = (t["entry_price"] - t["sl"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_utc, "reason": "SL", "pnl": pnl})
                print(f"[{to_ist(time_utc)}] | ❌ SELL Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue

            # TARGET bookings (T1–T5): use candle high/low (so intra-candle TP hitting counts)
            for idx in range(5):
                if t["booked"][idx] == 0:
                    if side == "BUY" and high >= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["targets"][idx] - t["entry_price"]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_utc, "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{to_ist(time_utc)}] | 🎯 BUY TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")
                    elif side == "SELL" and low <= t["targets"][idx]:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        if book_qty > t["remaining_qty"]:
                            book_qty = t["remaining_qty"]
                        if book_qty > 0:
                            pnl = (t["entry_price"] - t["targets"][idx]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": t["targets"][idx], "exit_time": time_utc, "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{to_ist(time_utc)}] | 🎯 SELL TP{idx+1} hit @ {t['targets'][idx]:.2f} | Remaining {t['remaining_qty']:.5f}")
    return trades, open_trade

# ----------------------------
# === Testnet order placer: 6 parts, rounds price & qty properly ===
# ----------------------------
def place_testnet_6part(side, entry_price, or_mid, or_range, trade_qty):
    print("📦 Placing testnet 6-part bracket orders...")
    fractions = [0.15, 0.20, 0.25, 0.20, 0.10, 0.10]  # as you decided
    multipliers = [2,4,6,8,10]
    for i in range(6):
        sub_qty = round_qty_step(trade_qty * fractions[i])
        if sub_qty <= 0:
            print(f"⚠️ sub_qty for part {i+1} evaluated to zero; skipping")
            continue

        # Market entry
        try:
            if side == "BUY":
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_BUY, type=ORDER_TYPE_MARKET, quantity=sub_qty)
            else:
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_SELL, type=ORDER_TYPE_MARKET, quantity=sub_qty)
        except Exception as e:
            print(f"⚠️ Error placing market entry for part {i+1}: {e}")

        # Compute TP price for parts 1..5
        if i < 5:
            if side == "BUY":
                tp_raw = entry_price + (or_range * multipliers[i])
            else:
                tp_raw = entry_price - (or_range * multipliers[i])
            tp_price = round_price_to_tick(tp_raw)
        else:
            tp_price = None

        sl_price = round_price_to_tick(or_mid)

        # Place TP limit (reduceOnly) for parts 1..5
        if tp_price is not None:
            try:
                if side == "BUY":
                    test_client.futures_create_order(symbol=SYMBOL, side=SIDE_SELL, type=ORDER_TYPE_LIMIT,
                                                    timeInForce=TIME_IN_FORCE_GTC, price=tp_price, quantity=sub_qty, reduceOnly=True)
                else:
                    test_client.futures_create_order(symbol=SYMBOL, side=SIDE_BUY, type=ORDER_TYPE_LIMIT,
                                                    timeInForce=TIME_IN_FORCE_GTC, price=tp_price, quantity=sub_qty, reduceOnly=True)
            except Exception as e:
                print(f"⚠️ Error placing TP (part {i+1}): {e}")

        # Place SL stop-market (reduceOnly) for each part
        try:
            if side == "BUY":
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_SELL, type="STOP_MARKET",
                                                stopPrice=sl_price, closePosition=False, quantity=sub_qty, reduceOnly=True)
            else:
                test_client.futures_create_order(symbol=SYMBOL, side=SIDE_BUY, type="STOP_MARKET",
                                                stopPrice=sl_price, closePosition=False, quantity=sub_qty, reduceOnly=True)
        except Exception as e:
            print(f"⚠️ Error placing SL (part {i+1}): {e}")

        time.sleep(0.12)
    print("🎯 Testnet 6-part orders submitted.\n")

# ----------------------------
# === Testnet helpers: cancel all open orders & flatten position ===
# ----------------------------
def cancel_all_testnet_orders():
    try:
        test_client.futures_cancel_all_open_orders(symbol=SYMBOL)
        print("✅ Cancelled all testnet open orders.")
    except Exception as e:
        print("⚠️ Error cancelling testnet orders:", e)

def flatten_all_testnet_positions():
    # Get current positions and flatten the symbol position if any
    try:
        pos_info = test_client.futures_position_information(symbol=SYMBOL)
        # pos_info is a list; find the record for symbol
        for p in pos_info:
            if p.get("symbol") != SYMBOL:
                continue
            amt = float(p.get("positionAmt", 0))
            if abs(amt) > 0:
                side = SIDE_SELL if amt > 0 else SIDE_BUY
                qty = abs(amt)
                qty_r = round_qty_step(qty)
                try:
                    test_client.futures_create_order(symbol=SYMBOL, side=side, type=ORDER_TYPE_MARKET,
                                                    quantity=qty_r, reduceOnly=True)
                    print(f"✅ Flattened position by {side} {qty_r}")
                except Exception as e:
                    print("⚠️ Error flattening position:", e)
            break
    except Exception as e:
        print("⚠️ Error getting position info:", e)

# ----------------------------
# === Run one orchestrated cycle (reconstruct -> maybe place orders) ===
# ----------------------------
def run_cycle(state):
    """
    state: dict that holds runtime flags across cycles:
      - session_start_utc
      - trades_today
      - open_trade (restored or created)
      - testnet_orders_placed (bool)
      - last_reset_date (date when reset executed)
    """
    try:
        now_utc = dt.datetime.now(dt.timezone.utc)
        now_ist = dt.datetime.now(TZ)
        # fetch candles (mainnet futures)
        df = fetch_intraday_data(SYMBOL, CANDLE_INTERVAL, days=2)
        session_start_utc = state.get("session_start_utc") or get_session_start(now_utc)
        state["session_start_utc"] = session_start_utc
        df_today = df[df.index >= session_start_utc]

        # compute ORB
        try:
            or_high, or_low, or_mid = get_opening_range(df_today, session_start_utc)
        except Exception as e:
            print(f"[{now_ist.strftime('%Y-%m-%d %H:%M:%S %Z')}] ⏳ Waiting for ORB data: {e}")
            return state

        last_price = df_today["Close"].iloc[-1]
        # compute trade_qty using leverage and round to step
        leveraged_value = TRADE_VALUE_USDT * LEVERAGE
        trade_qty = round_qty_step(leveraged_value / last_price)

        # Print header summary
        print("\n" + "-"*60)
        print(f"[{now_ist.strftime('%Y-%m-%d %H:%M:%S %Z')}] RUN CYCLE")
        print(f"📊 ORB (reconstructed) | High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f} | Range {(or_high-or_low):.2f} | Reset: {SESSION_RESET_HOUR:02d}:00 IST")
        # reconstruct trades and restore open trade if present
        trades_today, open_trade = reconstruct_trades_after_or(df_today, or_high, or_low, or_mid, trade_qty)
        state["trades_today"] = trades_today
        state["open_trade"] = open_trade

        # closed PnL
        closed_pnl = sum(t.get("pnl",0) for t in trades_today)

        # compute open PnL (if open_trade)
        open_pnl = 0.0
        if open_trade:
            if open_trade["side"] == "BUY":
                open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
            else:
                open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]

        print(f"💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {(closed_pnl+open_pnl):.2f}")

        # ---------- EOD reset handling ----------
        # Reset occurs at IST SESSION_RESET_HOUR:00 (user requested reset at 05:00 IST)
        # We only run reset once per calendar day when that timestamp occurs.
        last_reset_date = state.get("last_reset_date")  # date object in IST
        if now_ist.hour == SESSION_RESET_HOUR and now_ist.minute == 0:
            # If we didn't already reset for this IST date, perform reset now
            today_date = now_ist.date()
            if last_reset_date != today_date:
                print("🕔 EOD Reset triggered — cancelling orders & flattening testnet positions.")
                cancel_all_testnet_orders()
                flatten_all_testnet_positions()
                state["testnet_orders_placed"] = False
                state["last_reset_date"] = today_date
                # After reset, reconstruct again next cycle will find no open trade
                return state

        # ---------- If open trade exists -> skip new entries ----------
        
        if open_trade:
            print("⏳ Open trade detected from reconstruction. Skipping new entry placement.")
            # Keep state testnet_orders_placed consistent: if there are open positions on testnet we avoid double placing
            # If testnet_orders_placed is False but open_trade exists (restored from reconstruct), set True (safety).
            if not state.get("testnet_orders_placed", False):
                state["testnet_orders_placed"] = True
            return state

        # ---------- No open trade -> check for breakout and place testnet orders ----------
        if not state.get("testnet_orders_placed", False):
            # last candle index must be a completed 5-min candle (we are aligned on system clock)
            last_idx = df_today.index[-1]
            if last_idx.minute % 5 != 0:
                print("⏳ Last candle is not a completed 5-min candle yet; waiting for next cycle.")
                return state

            if last_price > or_high:
                print(f"[{to_ist(last_idx)}] | 📈 TRUE BREAKOUT BUY @ {last_price:.2f}")
                X = (or_high - or_low) / 2
                trade_qty_live = round_qty_step(leveraged_value / last_price)
                place_testnet_6part("BUY", last_price, or_mid, X, trade_qty_live)
                # create in-memory open trade placeholder so subsequent cycles won't place again until reset/close
                state["open_trade"] = {"side":"BUY","entry_price":last_price,"entry_time":last_idx,"sl":or_mid,"qty":trade_qty_live,"remaining_qty":trade_qty_live}
                state["testnet_orders_placed"] = True
                return state

            elif last_price < or_low:
                print(f"[{to_ist(last_idx)}] | 📉 TRUE BREAKOUT SELL @ {last_price:.2f}")
                X = (or_high - or_low) / 2
                trade_qty_live = round_qty_step(leveraged_value / last_price)
                place_testnet_6part("SELL", last_price, or_mid, X, trade_qty_live)
                state["open_trade"] = {"side":"SELL","entry_price":last_price,"entry_time":last_idx,"sl":or_mid,"qty":trade_qty_live,"remaining_qty":trade_qty_live}
                state["testnet_orders_placed"] = True
                return state
            else:
                print("⏳ No breakout this cycle.")
        else:
            print("⏳ Testnet orders already placed for current session / trade — skipping placement.")
        return state

    except Exception as e:
        print("⚠️ Error in run_cycle:", e)
        return state

# ----------------------------
# === Scheduler: run aligned to system 5-minute marks ===
# ----------------------------
def wait_until_next_5min_aligned():
    """Sleep until the next minute where minute % 5 == 0 and seconds==0 (system clock)."""
    while True:
        now = dt.datetime.now()
        sec = now.second
        minute = now.minute
        # compute seconds until next 5-min boundary aligned to top of minute
        next_min = ((minute // 5) + 1) * 5
        if next_min >= 60:
            # next hour
            next_dt = now.replace(hour=(now.hour+1)%24, minute=0, second=0, microsecond=0) + dt.timedelta(hours=0)
        else:
            next_dt = now.replace(minute=next_min, second=0, microsecond=0)
        delta = (next_dt - now).total_seconds()
        if delta <= 0:
            # if somehow negative or 0, run immediately
            return
        # sleep until that boundary
        time.sleep(delta)
        return

def production_loop():
    print("🚀 Production ORB v2 — running every 5 minutes aligned to system clock (:00, :05, :10 ...).")
    state = {
        "session_start_utc": None,
        "trades_today": [],
        "open_trade": None,
        "testnet_orders_placed": False,
        "last_reset_date": None
    }

    # run initial cycle immediately (but aligned to next 5-min mark per STEP1)
    wait_until_next_5min_aligned()
    while True:
        # Run one cycle
        state = run_cycle(state)
        # Wait for next aligned 5-minute boundary
        wait_until_next_5min_aligned()

# ----------------------------
if __name__ == "__main__":
    production_loop()


In [ ]:
#CODE WITH ORB HIGH LOW FOR TARGET CALCULATION

In [81]:
# final_orb_v2.py
# ============================================================
# FINAL ORB V2 — Live market-data (Binance Futures) + Testnet order placement
# ============================================================
import os
import time
import math
import pandas as pd
import datetime as dt
import pytz
from binance.client import Client
from binance.enums import *

# ----------------------------
# === CONFIGURATION ===
# ----------------------------
# API keys
LIVE_API_KEY = os.getenv("BINANCE_LIVE_API_KEY", "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM")
LIVE_API_SECRET = os.getenv("BINANCE_LIVE_API_SECRET", "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb")

TEST_API_KEY = os.getenv("BINANCE_TEST_API_KEY", "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm")
TEST_API_SECRET = os.getenv("BINANCE_TEST_API_SECRET", "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf")

# Symbol: user wants BTCUSDT_PERP naming — we'll accept that but map to API symbol when required
SYMBOL = "BTCUSDT_PERP"   # displayed variable; mapped for API (see below)
API_SYMBOL = SYMBOL.replace("_PERP", "")  # actual symbol for Binance futures API calls

# ORB & timing
CANDLE_INTERVAL = "5m"            # we check entries on completed 5m candles
OPENING_RANGE_MINS = 30           # OR window length (5:30 - 6:00 IST)
SESSION_RESET_HOUR = 5            # EOD reset at 05:00 IST
TZ = pytz.timezone("Asia/Kolkata")

# Capital & leverage
BASE_USDT = 1000                  # (per your last instruction)
LEVERAGE = 20                     # default leverage
DEMO_MODE = False                  # True => immediate placement (demo); False => wait for live breakout
SLEEP_SECONDS = 20                # main loop sleep (we still align to 5m boundary for entries)

# Safety / minor toggles
USE_MARK_PRICE_FOR_TRIGGER = True  # use mark price when checking breakout trigger (recommended)

# ----------------------------
# === CLIENTS ===
# ----------------------------
live_client = Client(LIVE_API_KEY, LIVE_API_SECRET)           # used for market data (futures endpoints)
test_client = Client(TEST_API_KEY, TEST_API_SECRET, testnet=True)  # used for order placement on testnet

# ----------------------------
# === Helpers: exchange filters & rounding ===
# ----------------------------
def get_symbol_filters(client, api_symbol):
    """Fetch price tick & lot step from futures exchange info"""
    info = client.futures_exchange_info()
    s = next((x for x in info["symbols"] if x["symbol"] == api_symbol), None)
    if s is None:
        raise RuntimeError(f"Symbol {api_symbol} not found in exchange info")
    price_filter = next(f for f in s["filters"] if f["filterType"] == "PRICE_FILTER")
    lot_size = next(f for f in s["filters"] if f["filterType"] == "LOT_SIZE")
    min_notional = next((f["notional"], None) for f in s["filters"] if f.get("filterType") == "MARKET_LOT_SIZE") if False else None
    tick = float(price_filter["tickSize"])
    step = float(lot_size["stepSize"])
    # compute decimal places
    def precision_from_step(x):
        s = "{:.8f}".format(x).rstrip('0')
        if '.' in s:
            return max(0, len(s.split('.')[1]))
        return 0
    return {"tick": tick, "step": step, "price_prec": precision_from_step(tick), "qty_prec": precision_from_step(step)}

symbol_filters = get_symbol_filters(test_client, API_SYMBOL)

def round_price_to_tick(p):
    tick = symbol_filters["tick"]
    prec = symbol_filters["price_prec"]
    # round to nearest tick
    rounded = math.floor(p / tick + 0.5) * tick
    return round(rounded, prec)

def round_qty_step(q):
    step = symbol_filters["step"]
    prec = symbol_filters["qty_prec"]
    if q <= 0:
        return round(step, prec)
    rounded = math.floor(q / step + 0.5) * step
    rounded = max(rounded, step)
    return round(rounded, prec)

# ----------------------------
# === Time helpers ===
# ----------------------------
def now_utc():
    return dt.datetime.now(dt.timezone.utc)

def to_ist_str(ts):
    """Accept tz-aware timestamp or pandas Timestamp (UTC), return IST string"""
    try:
        return pd.to_datetime(ts).tz_convert(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    except Exception:
        try:
            t = pd.to_datetime(ts)
            if t.tzinfo is None:
                t = t.tz_localize(dt.timezone.utc)
            return t.tz_convert(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
        except Exception:
            return str(ts)

# ----------------------------
# === Data fetcher (Binance Futures klines) ===
# ----------------------------
def fetch_intraday_data(api_symbol, interval, days=2):
    # limit calculation: safe number of candles = days*24*60 / interval_mins
    int_m = int(interval.replace('m',''))
    limit = max(200, min(1500, (days*24*60)//int_m + 10))
    klines = live_client.futures_klines(symbol=api_symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        "Open time","Open","High","Low","Close","Volume",
        "Close time","Quote asset volume","Number of trades",
        "Taker buy base","Taker buy quote","Ignore"
    ])
    df["Open time"] = pd.to_datetime(df["Open time"], unit="ms", utc=True)
    df = df.set_index("Open time")  # tz-aware UTC index
    df[["Open","High","Low","Close","Volume"]] = df[["Open","High","Low","Close","Volume"]].astype(float)
    return df

# ----------------------------
# === Session start / OR window in UTC ===
# ----------------------------
def get_session_start_utc(now_utc=None):
    # session start = UTC midnight of session day (00:00 UTC = 05:30 IST)
    now_utc = now_utc or now_utc() if False else (now_utc or dt.datetime.now(dt.timezone.utc))
    session_start_utc = now_utc.replace(hour=0, minute=0, second=0, microsecond=0)
    # If current UTC < 00:00 (not possible), subtract day
    return session_start_utc

def get_opening_range(df, session_start_utc):
    start = session_start_utc  # 00:00 UTC
    end = start + dt.timedelta(minutes=OPENING_RANGE_MINS)  # +30 => 00:30 UTC
    or_df = df[(df.index >= start) & (df.index < end)]
    if or_df.empty:
        raise ValueError("Opening range window empty.")
    or_high = or_df["High"].max()
    or_low = or_df["Low"].min()
    or_mid = (or_high + or_low) / 2.0
    # round to tick for consistency
    or_high = round_price_to_tick(or_high)
    or_low = round_price_to_tick(or_low)
    or_mid = round_price_to_tick(or_mid)
    return or_high, or_low, or_mid

# ----------------------------
# === Reconstruct trades (unchanged core logic) ===
# - Uses candle highs/lows for detecting TP & SL hits
# ----------------------------
def reconstruct_trades_after_or(df, or_high, or_low, or_mid, trade_qty):
    trades = []
    open_trade = None
    range_size = or_high - or_low
    X = range_size / 2.0
    # fraction per part (6 parts)
    fractions = [0.1, 0.15, 0.15, 0.20, 0.20, 0.20]
    multipliers = [2,4,6,8,10]  # T1..T5 (uses X * multipliers)

    # iterate through candles (we check only completed 5-min candles for entry/TP)
    for i in range(1, len(df)):
        close = df["Close"].iloc[i]
        high = df["High"].iloc[i]
        low = df["Low"].iloc[i]
        time_utc = df.index[i]  # tz-aware UTC

        # only consider completed 5-minute candle timestamps (minute divisible by 5)
        if time_utc.minute % 5 != 0:
            continue

        # ENTRY (if not in trade)
        if open_trade is None:
            # BUY breakout (close/candle mark moved from inside to > or_high)
            if close > or_high:
                entry = close
                sl = or_mid
                open_trade = {
                    "side": "BUY", "entry_price": entry, "entry_time": time_utc,
                    "sl": sl, "qty": trade_qty, "remaining_qty": trade_qty,
                    "booked": [0]*6
                }
                # compute TPs from ORB HIGH (per your new rule)
                open_trade["targets"] = [ round_price_to_tick(or_high + (X * m)) for m in multipliers ]
                print(f"[{to_ist_str(time_utc)}] | 📈 BUY triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets (from OR high):")
                for j, tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

            # SELL breakout
            if close < or_low:
                entry = close
                sl = or_mid
                open_trade = {
                    "side": "SELL", "entry_price": entry, "entry_time": time_utc,
                    "sl": sl, "qty": trade_qty, "remaining_qty": trade_qty,
                    "booked": [0]*6
                }
                # compute TPs from ORB LOW
                open_trade["targets"] = [ round_price_to_tick(or_low - (X * m)) for m in multipliers ]
                print(f"[{to_ist_str(time_utc)}] | 📉 SELL triggered @ {entry:.2f} | SL {sl:.2f}")
                print("🎯 Targets (from OR low):")
                for j, tgt in enumerate(open_trade["targets"], start=1):
                    print(f"   → TP{j}: {tgt:.2f}")
                continue

        # if in trade => manage exits
        if open_trade:
            t = open_trade
            side = t["side"]

            # STOPLOSS — use candle low/high hitting OR mid
            if side == "BUY" and low <= t["sl"]:
                pnl = (t["sl"] - t["entry_price"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_utc, "reason": "SL", "pnl": pnl})
                print(f"[{to_ist_str(time_utc)}] | ❌ BUY Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue
            if side == "SELL" and high >= t["sl"]:
                pnl = (t["entry_price"] - t["sl"]) * t["remaining_qty"]
                trades.append({**t, "exit_price": t["sl"], "exit_time": time_utc, "reason": "SL", "pnl": pnl})
                print(f"[{to_ist_str(time_utc)}] | ❌ SELL Stoploss hit @ {t['sl']:.2f}")
                open_trade = None
                continue

            # TARGET bookings T1..T5, using candle high/low
            for idx in range(5):
                if t["booked"][idx] == 0:
                    target_price = t["targets"][idx]
                    if side == "BUY" and high >= target_price:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        book_qty = min(book_qty, t["remaining_qty"])
                        if book_qty > 0:
                            pnl = (target_price - t["entry_price"]) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": target_price, "exit_time": time_utc,
                                           "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{to_ist_str(time_utc)}] | 🎯 BUY TP{idx+1} hit @ {target_price:.2f} | Remaining {t['remaining_qty']:.5f}")
                    elif side == "SELL" and low <= target_price:
                        book_qty = round(t["qty"] * fractions[idx], 5)
                        book_qty = min(book_qty, t["remaining_qty"])
                        if book_qty > 0:
                            pnl = (t["entry_price"] - target_price) * book_qty
                            t["remaining_qty"] -= book_qty
                            t["booked"][idx] = 1
                            trades.append({**t, "exit_price": target_price, "exit_time": time_utc,
                                           "reason": f"TP{idx+1}", "pnl": pnl, "qty": book_qty})
                            print(f"[{to_ist_str(time_utc)}] | 🎯 SELL TP{idx+1} hit @ {target_price:.2f} | Remaining {t['remaining_qty']:.5f}")

    return trades, open_trade

# ----------------------------
# === Testnet order placement: 6 parts bracket (market entries + TP limits + SL stop-market) ===
# - Uses ORB-high/low based TP calculation (passed in as or_range param: X = (or_high - or_low)/2)
# - SKIPS TP part if entry already beyond that TP (per your spec)
# ----------------------------
def place_testnet_6part(side, entry_price, or_mid, X, trade_qty, demo_mode=True):
    print("📦 Placing testnet 6-part bracket orders...")
    # fractions as requested
    fractions = [0.15, 0.20, 0.25, 0.20, 0.10, 0.10]
    multipliers = [2,4,6,8,10]  # T1..T5 - price = OR_high + X*m OR OR_low - X*m when SELL

    placed_orders = []
    # For entry we place a market order for entire qty parts individually (like earlier)
    for i in range(6):
        sub_qty_raw = trade_qty * fractions[i]
        sub_qty = round_qty_step(sub_qty_raw)
        if sub_qty <= 0:
            print(f"⚠️ part {i+1} qty is zero after rounding; skipping")
            continue

        # compute TP price (only for parts 1..5)
        if side == "BUY":
            tp_raw = entry_price + (X * multipliers[i]) if i < 5 else None
        else:
            tp_raw = entry_price - (X * multipliers[i]) if i < 5 else None

        tp_price = round_price_to_tick(tp_raw) if tp_raw is not None else None
        sl_price = round_price_to_tick(or_mid)

        # SKIP TP placement if entry is already beyond TP1 (or general TP) logic:
        skip_tp = False
        if tp_price is not None:
            if side == "BUY" and entry_price >= tp_price:
                skip_tp = True
            if side == "SELL" and entry_price <= tp_price:
                skip_tp = True

        # DEMO_MODE: immediate placement; else only return order data (simulate)
        if demo_mode:
            try:
                # Market entry
                if side == "BUY":
                    test_client.futures_create_order(symbol=API_SYMBOL, side=SIDE_BUY, type=ORDER_TYPE_MARKET, quantity=sub_qty)
                else:
                    test_client.futures_create_order(symbol=API_SYMBOL, side=SIDE_SELL, type=ORDER_TYPE_MARKET, quantity=sub_qty)
                placed_orders.append({"part": i+1, "side": side, "qty": sub_qty, "entry": "MARKET"})
            except Exception as e:
                print(f"⚠️ Error placing market entry for part {i+1}: {e}")

            # Place TP limit if not skipped (parts 1..5)
            if tp_price is not None and not skip_tp:
                try:
                    if side == "BUY":
                        test_client.futures_create_order(
                            symbol=API_SYMBOL,
                            side=SIDE_SELL,
                            type=ORDER_TYPE_LIMIT,
                            timeInForce=TIME_IN_FORCE_GTC,
                            price=tp_price,
                            quantity=sub_qty,
                            reduceOnly=True
                        )
                    else:
                        test_client.futures_create_order(
                            symbol=API_SYMBOL,
                            side=SIDE_BUY,
                            type=ORDER_TYPE_LIMIT,
                            timeInForce=TIME_IN_FORCE_GTC,
                            price=tp_price,
                            quantity=sub_qty,
                            reduceOnly=True
                        )
                    placed_orders[-1].update({"tp": tp_price})
                except Exception as e:
                    print(f"⚠️ Error placing TP (part {i+1}): {e}")

            # Place SL stop-market for each part
            try:
                # STOP_MARKET on Binance futures API is passed as type='STOP_MARKET' and stopPrice
                if side == "BUY":
                    test_client.futures_create_order(
                        symbol=API_SYMBOL,
                        side=SIDE_SELL,
                        type="STOP_MARKET",
                        stopPrice=sl_price,
                        closePosition=False,
                        quantity=sub_qty,
                        reduceOnly=True
                    )
                else:
                    test_client.futures_create_order(
                        symbol=API_SYMBOL,
                        side=SIDE_BUY,
                        type="STOP_MARKET",
                        stopPrice=sl_price,
                        closePosition=False,
                        quantity=sub_qty,
                        reduceOnly=True
                    )
                placed_orders[-1].update({"sl": sl_price})
            except Exception as e:
                print(f"⚠️ Error placing SL (part {i+1}): {e}")

            # tiny wait to avoid hitting rate limits
            time.sleep(0.12)

        else:
            # simulation path: don't call API, just report what would be placed
            item = {"part": i+1, "side": side, "qty": sub_qty, "tp": tp_price if not skip_tp else None, "sl": sl_price}
            placed_orders.append(item)

    print("🎯 Testnet 6-part placement complete.")
    return placed_orders

# ----------------------------
# === Utility: compute trade_qty from USDT and leverage ===
# ----------------------------
def qty_from_usdt_and_leverage(usdt_amount, last_price, leverage):
    effective = usdt_amount * leverage
    qty = effective / last_price
    qty = round_qty_step(qty)
    return qty

# ----------------------------
# === MAIN: reconstruct history then live monitor and place orders ===
# ----------------------------
def run_final_orb_v2(demo_mode=DEMO_MODE):
    print("🚀 Starting final_orb_v2 — ORB Live (mainnet futures data) + Testnet bracket orders")
    print(f"SYMBOL: {SYMBOL} (API: {API_SYMBOL}) | Base USDT: {BASE_USDT} | Leverage: {LEVERAGE}x | DEMO_MODE: {demo_mode}")
    # fetch candles
    df = fetch_intraday_data(API_SYMBOL, CANDLE_INTERVAL, days=2)
    session_start_utc = get_session_start_utc()
    df_today = df[df.index >= session_start_utc]

    # compute ORB
    try:
        or_high, or_low, or_mid = get_opening_range(df_today, session_start_utc)
    except Exception as e:
        print("⚠️ Opening range error:", e)
        return

    last_price = df_today["Close"].iloc[-1]
    trade_qty = qty_from_usdt_and_leverage(BASE_USDT, last_price, LEVERAGE)

    # print ORB summary
    ist_start = session_start_utc.astimezone(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    ist_end = (session_start_utc + dt.timedelta(minutes=OPENING_RANGE_MINS)).astimezone(TZ).strftime("%Y-%m-%d %H:%M:%S %Z")
    print(f"\n📊 Opening Range (reconstructed & rounded):\n  OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
    print(f"  OR Window (IST): {ist_start} → {ist_end}")
    print(f"  Using effective position size: {BASE_USDT} * {LEVERAGE} = {BASE_USDT * LEVERAGE} USDT -> Qty ~ {trade_qty}\n")

    # Reconstruct today's trades to get closed history and possible open_trade
    trades_today, open_trade = reconstruct_trades_after_or(df_today, or_high, or_low, or_mid, trade_qty)

    # print closed trades (simple)
    if trades_today:
        closed_df = pd.DataFrame(trades_today)

        # format timestamps
        closed_df["entry_time"] = closed_df["entry_time"].apply(lambda x: to_ist_str(x))
        closed_df["exit_time"] = closed_df["exit_time"].apply(lambda x: to_ist_str(x))

        # keep only relevant columns
        show_cols = ["side", "entry_price", "exit_price", "reason", "pnl", "qty", "entry_time", "exit_time"]
        closed_df = closed_df[show_cols]

        # round numeric columns
        closed_df["entry_price"] = closed_df["entry_price"].round(2)
        closed_df["exit_price"] = closed_df["exit_price"].round(2)
        closed_df["pnl"] = closed_df["pnl"].round(4)
        closed_df["qty"] = closed_df["qty"].round(4)

        print("\n📜 Closed trades (today):")
        for i, row in closed_df.iterrows():
            emoji = "🎯" if "TP" in row["reason"] else "❌" if "SL" in row["reason"] else "ℹ️"
            print(
                f"{emoji} {row['side']:>4} | "
                f"Entry: {row['entry_price']:.2f} | Exit: {row['exit_price']:.2f} | "
                f"Reason: {row['reason']:<3} | Qty: {row['qty']:.4f} | "
                f"PnL: {row['pnl']:+.2f} | "
                f"Entry: {row['entry_time']} | Exit: {row['exit_time']}"
            )
        print()
    else:
        print("📜 Closed trades (today): None")

    closed_pnl = sum(t.get("pnl", 0) for t in trades_today)

    # if open trade restored, compute open pnl and avoid placing new trades
    if open_trade:
        last_price = df_today["Close"].iloc[-1]
        if open_trade["side"] == "BUY":
            open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
        else:
            open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]
        total_pnl = closed_pnl + open_pnl
        print(f"\n🔔 Open trade restored: Side {open_trade['side']} | Entry {open_trade['entry_price']:.2f} | SL {open_trade['sl']:.2f} | RemainingQty {open_trade['remaining_qty']:.5f}")
        print(f"💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {total_pnl:.2f}\n")
    else:
        print(f"\n💰 Closed PnL: {closed_pnl:.2f} | Open PnL: 0.00 | Total: {closed_pnl:.2f}\n")

    # LIVE MONITOR LOOP
    testnet_orders_placed = False
    while True:
        try:
            # alignment: if current system minute not changed, sleep tiny bit
            now_utc = dt.datetime.now(dt.timezone.utc)
            now_ist = now_utc.astimezone(TZ)
            # fetch fresh candles each loop
            df = fetch_intraday_data(API_SYMBOL, CANDLE_INTERVAL, days=2)
            df_today = df[df.index >= session_start_utc]
            last_idx = df_today.index[-1]
            last_close = df_today["Close"].iloc[-1]

            # print heartbeat
            print(f"[{to_ist_str(last_idx)}] | Current price: {last_close:.2f}")

            # check if session rolled (EOD reset)
            if now_utc.date() != session_start_utc.date():
                print("🕔 Session date changed — performing EOD reset (clearing open trade/pending flags).")
                session_start_utc = get_session_start_utc(now_utc)
                df_today = df[df.index >= session_start_utc]
                # recompute ORB for new session
                try:
                    or_high, or_low, or_mid = get_opening_range(df_today, session_start_utc)
                    print(f"🔁 New ORB computed: OR High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
                except Exception as e:
                    print("⚠️ ORB recompute error:", e)
                open_trade = None
                testnet_orders_placed = False
                trades_today = []
                closed_pnl = 0.0

            # if an open_trade exists from reconstruction or earlier live placement, just compute PnL and skip placement
            if open_trade:
                last_price = last_close
                if open_trade["side"] == "BUY":
                    open_pnl = (last_price - open_trade["entry_price"]) * open_trade["remaining_qty"]
                else:
                    open_pnl = (open_trade["entry_price"] - last_price) * open_trade["remaining_qty"]
                closed_pnl = sum(t.get("pnl",0) for t in trades_today)
                total_pnl = closed_pnl + open_pnl
                print(f"💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {total_pnl:.2f}")
                # wait until next minute
                time.sleep(SLEEP_SECONDS)
                continue

            # only attempt entry logic at completed 5-min candle timestamps (minute % 5 == 0)
            if last_idx.minute % 5 != 0:
                time.sleep(5)
                continue

            # Determine breakout trigger using mark price if requested
            if USE_MARK_PRICE_FOR_TRIGGER:
                try:
                    mark = live_client.futures_mark_price(symbol=API_SYMBOL)
                    mark_price = float(mark.get("markPrice", last_close))
                except Exception:
                    mark_price = last_close
            else:
                mark_price = last_close

            # Use ORB-high/low derived TPs (X)
            X = (or_high - or_low) / 2.0

            # If breakout and we have no open trade place testnet orders (or simulate if demo_mode False)
            if mark_price > or_high and not testnet_orders_placed:
                print(f"[{to_ist_str(last_idx)}] | 📈 TRUE BREAKOUT BUY (MARK: {mark_price:.2f}) — preparing orders")
                # recalc trade_qty to reflect latest price
                trade_qty_live = qty_from_usdt_and_leverage(BASE_USDT, mark_price, LEVERAGE)
                placed = place_testnet_6part("BUY", or_high, or_mid, X, trade_qty_live, demo_mode=demo_mode)
                # create in-memory open_trade to prevent duplicates
                open_trade = {"side":"BUY", "entry_price": mark_price, "entry_time": last_idx, "sl": or_mid, "qty": trade_qty_live, "remaining_qty": trade_qty_live}
                trades_today.extend([])  # keep earlier trades
                testnet_orders_placed = True
                print("✅ Orders placed (or simulated).")

            elif mark_price < or_low and not testnet_orders_placed:
                print(f"[{to_ist_str(last_idx)}] | 📉 TRUE BREAKOUT SELL (MARK: {mark_price:.2f}) — preparing orders")
                trade_qty_live = qty_from_usdt_and_leverage(BASE_USDT, mark_price, LEVERAGE)
                placed = place_testnet_6part("SELL", or_low, or_mid, X, trade_qty_live, demo_mode=demo_mode)
                open_trade = {"side":"SELL", "entry_price": mark_price, "entry_time": last_idx, "sl": or_mid, "qty": trade_qty_live, "remaining_qty": trade_qty_live}
                testnet_orders_placed = True
                print("✅ Orders placed (or simulated).")

            # print quick summary each loop
            closed_pnl = sum(t.get("pnl",0) for t in trades_today)
            open_pnl = 0.0
            if open_trade:
                if open_trade["side"] == "BUY":
                    open_pnl = (last_close - open_trade["entry_price"]) * open_trade["remaining_qty"]
                else:
                    open_pnl = (open_trade["entry_price"] - last_close) * open_trade["remaining_qty"]
            print(f"💰 Closed PnL: {closed_pnl:.2f} | Open PnL: {open_pnl:.2f} | Total: {closed_pnl + open_pnl:.2f} | Time: {to_ist_str(last_idx)}")

        except Exception as e:
            print("⚠️ Runtime error in loop:", e)

        # sleep to reduce API calls; we still align to 5-min candles by checking minute%5 above
        time.sleep(SLEEP_SECONDS)

# ----------------------------
# Entry point
# ----------------------------
if __name__ == "__main__":
    # You can toggle demo mode here
    run_final_orb_v2(demo_mode=DEMO_MODE)


🚀 Starting final_orb_v2 — ORB Live (mainnet futures data) + Testnet bracket orders
SYMBOL: BTCUSDT_PERP (API: BTCUSDT) | Base USDT: 1000 | Leverage: 20x | DEMO_MODE: False

📊 Opening Range (reconstructed & rounded):
  OR High 103840.70 | Low 103264.60 | Mid 103552.60
  OR Window (IST): 2025-11-06 05:30:00 IST → 2025-11-06 06:00:00 IST
  Using effective position size: 1000 * 20 = 20000 USDT -> Qty ~ 0.193

[2025-11-06 06:35:00 IST] | 📉 SELL triggered @ 103186.60 | SL 103552.60
🎯 Targets (from OR low):
   → TP1: 102688.50
   → TP2: 102112.40
   → TP3: 101536.30
   → TP4: 100960.20
   → TP5: 100384.10
[2025-11-06 06:55:00 IST] | 🎯 SELL TP1 hit @ 102688.50 | Remaining 0.17370
[2025-11-06 08:45:00 IST] | ❌ SELL Stoploss hit @ 103552.60

📜 Closed trades (today):
🎯 SELL | Entry: 103186.60 | Exit: 102688.50 | Reason: TP1 | Qty: 0.0193 | PnL: +9.61 | Entry: 2025-11-06 06:35:00 IST | Exit: 2025-11-06 06:55:00 IST
❌ SELL | Entry: 103186.60 | Exit: 103552.60 | Reason: SL  | Qty: 0.1930 | PnL: -63.

KeyboardInterrupt: 

In [52]:
import pandas as pd
import pytz
from datetime import datetime, timedelta
from binance.client import Client
from binance.enums import *
import time
import math

# ==== USER CONFIG ====


API_KEY = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
API_SECRET = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"

TESTNET_KEY = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
TESTNET_SECRET = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

symbol = "BTCUSDT"
symbol_display = "BTCUSDT_PERP"
DEMO_MODE = True  # ⬅️ True = immediate test order; False = live conditional logic
base_usdt = 1000
leverage = 20
orb_start = "05:30"
orb_end = "06:00"
reset_hour_ist = 5  # Exit all at 5AM IST

# ==== INIT ====
client_main = Client(API_KEY, API_SECRET)
client_test = Client(TESTNET_KEY, TESTNET_SECRET, testnet=True)

# Force One-Way mode
try:
    client_test.futures_change_position_mode(dualSidePosition=False)
    print("✅ Position mode set to ONE-WAY (dualSidePosition=False)")
except Exception as e:
    print(f"⚠️ Could not set position mode: {e}")

# Confirm
try:
    mode = client_test.futures_get_position_mode()
    print("🔍 Position Mode Status:", mode)
except:
    pass

# ==== GET PRECISION ====
def get_symbol_precision(client, symbol):
    info = client.futures_exchange_info()
    for s in info["symbols"]:
        if s["symbol"] == symbol:
            tick_size = float([f for f in s["filters"] if f["filterType"] == "PRICE_FILTER"][0]["tickSize"])
            step_size = float([f for f in s["filters"] if f["filterType"] == "LOT_SIZE"][0]["stepSize"])
            return tick_size, step_size
    raise Exception("Symbol not found")

tick_size, step_size = get_symbol_precision(client_main, symbol)

def round_price(p):
    return round(round(p / tick_size) * tick_size, int(abs(math.log10(tick_size))))

def round_qty(q):
    return round(round(q / step_size) * step_size, int(abs(math.log10(step_size))))

# ==== GET DATA ====
def get_futures_data(symbol, interval="1m", lookback="1 day ago UTC"):
    klines = client_main.futures_historical_klines(symbol, interval, lookback)
    df = pd.DataFrame(klines, columns=[
        "time","o","h","l","c","v","ct","qv","n","tb","tq","ig"
    ])
    df["time"] = pd.to_datetime(df["time"], unit="ms", utc=True)
    df[["o","h","l","c"]] = df[["o","h","l","c"]].astype(float)
    return df[["time","o","h","l","c"]]

# ==== ORB RANGE ====
def get_orb_range(df):
    ist = pytz.timezone("Asia/Kolkata")
    df["time_ist"] = df["time"].dt.tz_convert(ist)
    start = datetime.strptime(orb_start, "%H:%M").time()
    end = datetime.strptime(orb_end, "%H:%M").time()
    orb_df = df[(df["time_ist"].dt.time >= start) & (df["time_ist"].dt.time < end)]
    high = orb_df["h"].max()
    low = orb_df["l"].min()
    mid = (high + low) / 2
    print(f"\n📊 Opening Range (reconstructed):\n  OR High {high:.2f} | Low {low:.2f} | Mid {mid:.2f}")
    return high, low, mid

# ==== ORDER PLACEMENT ====
def place_bracket_orders_testnet(side, entry, sl, targets, qty):
    print(f"🚀 DEMO_MODE active — placing 6-part bracket order now (Testnet)...")

    # Split into 6 equal parts
    parts = [0.15, 0.15, 0.15, 0.15, 0.15, 0.25]
    for i, frac in enumerate(parts, start=1):
        part_qty = round_qty(qty * frac)
        tp = targets[i - 1] if i <= 5 else None
        try:
            if tp:
                # Take-profit order
                client_test.futures_create_order(
                    symbol=symbol,
                    side="SELL" if side == "BUY" else "BUY",
                    type="TAKE_PROFIT_MARKET",
                    stopPrice=round_price(tp),
                    closePosition=False,
                    quantity=part_qty,
                    timeInForce="GTC"
                )
                print(f"✅ TP{i} placed: {tp} ({part_qty})")
            else:
                print(f"✅ Part {i} has open TP (manual exit or reset).")

        except Exception as e:
            print(f"⚠️ Error placing order {i}: {e}")

    # Stoploss
    try:
        client_test.futures_create_order(
            symbol=symbol,
            side="SELL" if side == "BUY" else "BUY",
            type="STOP_MARKET",
            stopPrice=round_price(sl),
            closePosition=False,
            quantity=round_qty(qty),
            timeInForce="GTC"
        )
        print(f"✅ SL placed @ {sl}")
    except Exception as e:
        print(f"⚠️ Error placing SL: {e}")

    print("✅ Demo order placement complete — skipping live logic.\n")

# ==== MAIN ====
print(f"\n🚀 Starting final_orb_v2 — ORB Live (mainnet futures data) + Testnet bracket orders")
print(f"SYMBOL: {symbol_display} | Base USDT: {base_usdt} | Leverage: {leverage}x | DEMO_MODE: {DEMO_MODE}\n")

df = get_futures_data(symbol)
orb_high, orb_low, orb_mid = get_orb_range(df)

# Calculate qty
mark_price = float(client_main.futures_mark_price(symbol=symbol)["markPrice"])
position_value = base_usdt * leverage
qty = round_qty(position_value / mark_price)
print(f"  Using effective position size: {position_value} USDT -> Qty ≈ {qty}\n")

# Entry simulation (Buy breakout)
entry_price = mark_price
targets = [orb_high + (i + 1) * (orb_high - orb_low) for i in range(5)]
sl = orb_mid

print(f"[{datetime.now(pytz.timezone('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S %Z')}] | 📈 BUY triggered @ {entry_price:.2f} | SL {sl:.2f}")
print("🎯 Targets (from OR high):")
for i, t in enumerate(targets, 1):
    print(f"   → TP{i}: {t:.2f}")

# DEMO order placement
if DEMO_MODE:
    place_bracket_orders_testnet("BUY", entry_price, sl, targets, qty)
else:
    print("🕒 Live mode: Waiting for entry conditions...")


⚠️ Could not set position mode: APIError(code=-4059): No need to change position side.
🔍 Position Mode Status: {'dualSidePosition': False}

🚀 Starting final_orb_v2 — ORB Live (mainnet futures data) + Testnet bracket orders
SYMBOL: BTCUSDT_PERP | Base USDT: 1000 | Leverage: 20x | DEMO_MODE: True


📊 Opening Range (reconstructed):
  OR High 109600.00 | Low 109351.50 | Mid 109475.75
  Using effective position size: 20000 USDT -> Qty ≈ 0.182

[2025-11-01 14:59:11 IST] | 📈 BUY triggered @ 110128.00 | SL 109475.75
🎯 Targets (from OR high):
   → TP1: 109848.50
   → TP2: 110097.00
   → TP3: 110345.50
   → TP4: 110594.00
   → TP5: 110842.50
🚀 DEMO_MODE active — placing 6-part bracket order now (Testnet)...
⚠️ Error placing order 1: APIError(code=-2021): Order would immediately trigger.
⚠️ Error placing order 2: APIError(code=-2021): Order would immediately trigger.
✅ TP3 placed: 110345.5 (0.027)
✅ TP4 placed: 110594.0 (0.027)
✅ TP5 placed: 110842.5 (0.027)
✅ Part 6 has open TP (manual exit or r

In [65]:
# ===============================================
#  FINAL_ORB_V2.PY
#  Opening Range Breakout (Futures) — Live + Testnet Bracket Orders
#  Mode: mainnet data, testnet order placement
#  Updated: 2025-11-01
# ===============================================


# -----------------------------------------------
# CONFIGURATION
# -----------------------------------------------
# ✅ Replace with your actual keys (same as Final ORB v1)



import pandas as pd
import numpy as np
import time
import pytz
from datetime import datetime, timedelta
from binance.client import Client
from binance.enums import *
import math

# ==================== CONFIG ==================== #
API_KEY_MAIN = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
API_SECRET_MAIN = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"
API_KEY_TEST = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
API_SECRET_TEST = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"


symbol = "BTCUSDT_PERP"
binance_symbol = "BTCUSDT"

LEVERAGE = 20
BASE_USDT = 1000
DEMO_MODE = False

IST = pytz.timezone("Asia/Kolkata")
RESET_HOUR = 5  # reset trades at 5AM IST
OR_START = "05:30"
OR_END = "06:00"

# Connect clients
client_main = Client(API_KEY_MAIN, API_SECRET_MAIN)
client_test = Client(API_KEY_TEST, API_SECRET_TEST, testnet=True)

# ==================== HELPERS ==================== #
def get_futures_precision(symbol):
    info = client_main.futures_exchange_info()
    for s in info['symbols']:
        if s['symbol'] == binance_symbol:
            tick_size = float([f for f in s['filters'] if f['filterType'] == 'PRICE_FILTER'][0]['tickSize'])
            step_size = float([f for f in s['filters'] if f['filterType'] == 'LOT_SIZE'][0]['stepSize'])
            price_precision = int(round(-math.log(tick_size, 10), 0))
            qty_precision = int(round(-math.log(step_size, 10), 0))
            return price_precision, qty_precision
    return 2, 3

PRICE_PRECISION, QTY_PRECISION = get_futures_precision(binance_symbol)

def round_to_precision(value, precision):
    return float(round(value, precision))

def print_ist_time():
    return datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S IST")

# ==================== DATA FETCH ==================== #
def fetch_futures_klines(symbol, interval="1m", lookback="1 day ago UTC"):
    klines = client_main.futures_historical_klines(binance_symbol, interval, lookback)
    df = pd.DataFrame(klines, columns=['timestamp','open','high','low','close','volume','close_time',
                                       'qav','num_trades','tbbav','tbqav','ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.set_index('timestamp')
    df = df.astype(float)
    return df

# ==================== OPENING RANGE ==================== #
def get_opening_range(df):
    start_utc = datetime.now(pytz.utc).replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(hours=0)
    start = start_utc + timedelta(hours=5, minutes=30)  # 5:30 AM IST
    end = start + timedelta(minutes=30)
    df_range = df.between_time(start.time(), end.time())
    if df_range.empty:
        return None, None, None
    high = df_range['high'].max()
    low = df_range['low'].min()
    mid = (high + low) / 2
    return high, low, mid

# ==================== RECONSTRUCTION ==================== #
def reconstruct_trades(df, or_high, or_low, or_mid):
    trades = []
    position = None
    entry_price = sl = 0
    qty = round_to_precision((BASE_USDT * LEVERAGE) / df['close'].iloc[-1], QTY_PRECISION)
    parts = [0.15, 0.20, 0.25, 0.15, 0.15, 0.10]

    for i in range(1, len(df)):
        row_prev, row = df.iloc[i-1], df.iloc[i]

        # BUY entry
        if position is None and row_prev['close'] <= or_high and row['close'] > or_high:
            position = "BUY"
            entry_price = row['close']
            sl = or_mid
            range_ = or_high - or_low
            targets = [or_high + (i+1)*range_ for i in range(5)]
            trades.append(["BUY", entry_price, row.name, sl, qty, parts, targets, 0, None, None, None, 0])
            continue

        # EXIT (use candle high/low for TP hit)
        if position == "BUY":
            for trade in trades:
                if trade[9] is not None:
                    continue
                tps = trade[6]
                for j, tp in enumerate(tps):
                    if row['high'] >= tp:
                        trade[9] = tp
                        trade[10] = row.name
                        trade[11] = f"TP{j+1}"
                        trade[7] = round((tp - trade[1]) * qty / trade[1], 2)
                        position = None
                        break
                if row['low'] <= sl:
                    trade[9] = sl
                    trade[10] = row.name
                    trade[11] = "SL"
                    trade[7] = round((sl - trade[1]) * qty / trade[1], 2)
                    position = None
    df_trades = pd.DataFrame(trades, columns=[
        "side","entry_price","entry_time","sl","qty","parts","targets","pnl","exit_price","exit_time","reason","pnl_value"
    ])
    return df_trades

# ==================== ORDER PLACEMENT ==================== #
def place_bracket_orders(entry_price, sl, targets, qty, side="BUY"):
    parts = [0.15, 0.20, 0.25, 0.15, 0.15, 0.10]
    for i in range(6):
        part_qty = round(qty * parts[i], QTY_PRECISION)
        tp_price = round_to_precision(targets[i] if i < 5 else None, PRICE_PRECISION)
        try:
            if DEMO_MODE:
                print(f"[DEMO] Placing {side} order {i+1}: qty={part_qty}, TP={tp_price}, SL={sl}")
                continue
            # entry (market)
            client_test.futures_create_order(
                symbol=binance_symbol,
                side=SIDE_BUY if side=="BUY" else SIDE_SELL,
                type=ORDER_TYPE_MARKET,
                quantity=part_qty
            )
            # TP
            if i < 5:
                client_test.futures_create_order(
                    symbol=binance_symbol,
                    side=SIDE_SELL if side=="BUY" else SIDE_BUY,
                    type=ORDER_TYPE_LIMIT,
                    timeInForce="GTC",
                    quantity=part_qty,
                    price=tp_price,
                    reduceOnly=True
                )
            # SL
            client_test.futures_create_order(
                symbol=binance_symbol,
                side=SIDE_SELL if side=="BUY" else SIDE_BUY,
                type=ORDER_TYPE_STOP_MARKET,
                stopPrice=round_to_precision(sl, PRICE_PRECISION),
                closePosition=True
            )
        except Exception as e:
            print(f"⚠️ Error placing part {i+1}: {e}")

# ==================== MAIN LOOP ==================== #
def orb_main_loop():
    print(f"🚀 Starting final_orb_v2 — Live (mainnet futures data) + Testnet bracket orders")
    print(f"SYMBOL: {symbol} | Base USDT: {BASE_USDT} | Leverage: {LEVERAGE}x | DEMO_MODE: {DEMO_MODE}")
    print()

    while True:
        now = datetime.now(IST)
        if now.hour == RESET_HOUR and now.minute < 5:
            print(f"[{print_ist_time()}] 🔄 Daily reset triggered — closing all open trades...")
            try:
                client_test.futures_cancel_all_open_orders(symbol=binance_symbol)
                client_test.futures_create_order(
                    symbol=binance_symbol,
                    side=SIDE_SELL,
                    type=ORDER_TYPE_MARKET,
                    quantity=0.1,
                    reduceOnly=True
                )
            except Exception as e:
                print(f"⚠️ Reset error: {e}")

        # Run only at 5-minute marks
        if now.minute % 5 == 0 and now.second < 5:
            df = fetch_futures_klines(symbol=binance_symbol, interval="1m", lookback="12 hours ago UTC")
            or_high, or_low, or_mid = get_opening_range(df)
            if or_high is None:
                print("⚠️ OR not ready yet...")
                time.sleep(60)
                continue

            print(f"\n📊 Opening Range (IST): High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
            df_trades = reconstruct_trades(df, or_high, or_low, or_mid)
            if not df_trades.empty:
                print(df_trades.tail(3))
            else:
                print("No trades reconstructed yet.")

            # Check for fresh entry
            mark_price = float(client_main.futures_mark_price(symbol=binance_symbol)['markPrice'])
            if mark_price > or_high:
                range_ = or_high - or_low
                targets = [or_high + (i+1)*range_ for i in range(5)]
                qty = round_to_precision((BASE_USDT * LEVERAGE) / mark_price, QTY_PRECISION)
                print(f"\n[{print_ist_time()}] | 📈 BUY triggered @ {mark_price:.2f} | SL {or_mid:.2f}")
                place_bracket_orders(mark_price, or_mid, targets + [None], qty, side="BUY")
            elif mark_price < or_low:
                range_ = or_high - or_low
                targets = [or_low - (i+1)*range_ for i in range(5)]
                qty = round_to_precision((BASE_USDT * LEVERAGE) / mark_price, QTY_PRECISION)
                print(f"\n[{print_ist_time()}] | 📉 SELL triggered @ {mark_price:.2f} | SL {or_mid:.2f}")
                place_bracket_orders(mark_price, or_mid, targets + [None], qty, side="SELL")

        time.sleep(3)

# ==================== START ==================== #
if __name__ == "__main__":
    orb_main_loop()


🚀 Starting final_orb_v2 — Live (mainnet futures data) + Testnet bracket orders
SYMBOL: BTCUSDT_PERP | Base USDT: 1000 | Leverage: 20x | DEMO_MODE: False


📊 Opening Range (IST): High 110516.00 | Low 110230.00 | Mid 110373.00
  side  entry_price          entry_time        sl    qty  \
2  BUY     110559.6 2025-11-02 09:46:00  110373.0  0.182   
3  BUY     110517.2 2025-11-02 11:01:00  110373.0  0.182   
4  BUY     110556.5 2025-11-02 14:03:00  110373.0  0.182   

                                parts  \
2  [0.15, 0.2, 0.25, 0.15, 0.15, 0.1]   
3  [0.15, 0.2, 0.25, 0.15, 0.15, 0.1]   
4  [0.15, 0.2, 0.25, 0.15, 0.15, 0.1]   

                                             targets  pnl exit_price  \
2  [110802.0, 111088.0, 111374.0, 111660.0, 11194...  0.0       None   
3  [110802.0, 111088.0, 111374.0, 111660.0, 11194...  0.0       None   
4  [110802.0, 111088.0, 111374.0, 111660.0, 11194... -0.0       None   

   exit_time              reason pnl_value  
2   110802.0 2025-11-02 10:14:00   

TypeError: type NoneType doesn't define __round__ method

In [67]:
# ===============================================
#  FINAL_ORB_V2.PY
#  Opening Range Breakout (Futures) — Live + Testnet Bracket Orders
#  Mode: mainnet data, testnet order placement
#  Updated: 2025-11-02
# ===============================================

import pandas as pd
import numpy as np
import time
import pytz
from datetime import datetime, timedelta
from binance.client import Client
from binance.enums import *
import math

# -----------------------------------------------
# CONFIGURATION
# -----------------------------------------------
API_KEY_MAIN = "PNtc6nKWqFuXoUQM1cSZfdHAFdS5fA6sidGlVTdfN2C3qNXlUeLDHr2DEmqmPLWM"
API_SECRET_MAIN = "hXcuBhJafPGnI8rjOqSpjZkCkKQSurLQBwSS1jj83ucEpop75VrcBnN1z09tYfwb"
API_KEY_TEST = "jfMQBx4mhga6U2IvipMKt839Pg8JXtFx7mVp3QDNoZPltkM63OFEhYKs0QjRphQm"
API_SECRET_TEST = "pO4dO9FjSW4cvf39XofyJNzfKzol0WHBVpHhYGtiCIgDN7LEf3C0OK19BMeLd6Tf"

symbol = "BTCUSDT_PERP"
binance_symbol = "BTCUSDT"

LEVERAGE = 20
BASE_USDT = 1000
DEMO_MODE = False  # ✅ False means real logic (True = instant order placement demo)

IST = pytz.timezone("Asia/Kolkata")
RESET_HOUR = 5  # reset at 5AM IST
OR_START = "05:30"
OR_END = "06:00"

# -----------------------------------------------
# CLIENTS
# -----------------------------------------------
client_main = Client(API_KEY_MAIN, API_SECRET_MAIN)
client_test = Client(API_KEY_TEST, API_SECRET_TEST, testnet=True)

# -----------------------------------------------
# HELPERS
# -----------------------------------------------
def get_futures_precision(symbol):
    info = client_main.futures_exchange_info()
    for s in info['symbols']:
        if s['symbol'] == binance_symbol:
            tick_size = float([f for f in s['filters'] if f['filterType'] == 'PRICE_FILTER'][0]['tickSize'])
            step_size = float([f for f in s['filters'] if f['filterType'] == 'LOT_SIZE'][0]['stepSize'])
            price_precision = int(round(-math.log(tick_size, 10), 0))
            qty_precision = int(round(-math.log(step_size, 10), 0))
            return price_precision, qty_precision
    return 2, 3

PRICE_PRECISION, QTY_PRECISION = get_futures_precision(binance_symbol)

def round_to_precision(value, precision):
    return float(round(value, precision))

def print_ist_time():
    return datetime.now(IST).strftime("%Y-%m-%d %H:%M:%S IST")

# -----------------------------------------------
# FETCH DATA
# -----------------------------------------------
def fetch_futures_klines(symbol, interval="1m", lookback="1 day ago UTC"):
    klines = client_main.futures_historical_klines(binance_symbol, interval, lookback)
    df = pd.DataFrame(klines, columns=[
        'timestamp','open','high','low','close','volume','close_time',
        'qav','num_trades','tbbav','tbqav','ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.set_index('timestamp')
    df = df.astype(float)
    return df

# -----------------------------------------------
# OPENING RANGE CALCULATION (Binance time but print IST)
# -----------------------------------------------
def get_opening_range(df):
    now_utc = datetime.now(pytz.utc)
    or_start_utc = now_utc.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(hours=0) + timedelta(hours=5, minutes=30)
    or_end_utc = or_start_utc + timedelta(minutes=30)
    df_range = df.between_time(or_start_utc.time(), or_end_utc.time())
    if df_range.empty:
        return None, None, None
    high = df_range['high'].max()
    low = df_range['low'].min()
    mid = (high + low) / 2
    return high, low, mid

# -----------------------------------------------
# RECONSTRUCTION (includes candle high/low for TP)
# -----------------------------------------------
def reconstruct_trades(df, or_high, or_low, or_mid):
    trades = []
    position = None
    entry_price = sl = 0
    qty = round_to_precision((BASE_USDT * LEVERAGE) / df['close'].iloc[-1], QTY_PRECISION)
    parts = [0.15, 0.20, 0.25, 0.15, 0.15, 0.10]

    for i in range(1, len(df)):
        row_prev, row = df.iloc[i-1], df.iloc[i]

        if position is None and row_prev['close'] <= or_high and row['close'] > or_high:
            position = "BUY"
            entry_price = row['close']
            sl = or_mid
            range_ = or_high - or_low
            targets = [or_high + (i+1)*range_ for i in range(5)]
            trades.append(["BUY", entry_price, row.name, sl, qty, parts, targets, 0, None, None, None, 0])
            continue

        if position == "BUY":
            for trade in trades:
                if trade[9] is not None:
                    continue
                tps = trade[6]
                for j, tp in enumerate(tps):
                    if row['high'] >= tp:
                        trade[9] = tp
                        trade[10] = row.name
                        trade[11] = f"TP{j+1}"
                        trade[7] = round((tp - trade[1]) * qty / trade[1], 2)
                        position = None
                        break
                if row['low'] <= sl:
                    trade[9] = sl
                    trade[10] = row.name
                    trade[11] = "SL"
                    trade[7] = round((sl - trade[1]) * qty / trade[1], 2)
                    position = None
    df_trades = pd.DataFrame(trades, columns=[
        "side","entry_price","entry_time","sl","qty","parts","targets","pnl","exit_price","exit_time","reason","pnl_value"
    ])
    return df_trades

# -----------------------------------------------
# BRACKET ORDER PLACEMENT (6 parts)
# -----------------------------------------------
def place_bracket_orders(entry_price, sl, targets, qty, side="BUY"):
    parts = [0.15, 0.20, 0.25, 0.15, 0.15, 0.10]
    for i in range(6):
        part_qty = round(qty * parts[i], QTY_PRECISION)
        tp_price = targets[i] if i < 5 else None

        try:
            if DEMO_MODE:
                print(f"[DEMO] {side} order {i+1}: qty={part_qty}, TP={tp_price}, SL={sl}")
                continue

            # Entry (Market Order)
            client_test.futures_create_order(
                symbol=binance_symbol,
                side=SIDE_BUY if side == "BUY" else SIDE_SELL,
                type="MARKET",
                quantity=part_qty
            )

            # Take-Profit (LIMIT order)
            if i < 5:
                client_test.futures_create_order(
                    symbol=binance_symbol,
                    side=SIDE_SELL if side == "BUY" else SIDE_BUY,
                    type="LIMIT",
                    timeInForce="GTC",
                    quantity=part_qty,
                    price=round_to_precision(tp_price, PRICE_PRECISION),
                    reduceOnly=True
                )

            # Stop-Loss (STOP_MARKET order)
            client_test.futures_create_order(
                symbol=binance_symbol,
                side=SIDE_SELL if side == "BUY" else SIDE_BUY,
                type="STOP_MARKET",
                stopPrice=round_to_precision(sl, PRICE_PRECISION),
                closePosition=True
            )

        except Exception as e:
            print(f"⚠️ Error placing part {i+1}: {e}")


# -----------------------------------------------
# MAIN LOOP
# -----------------------------------------------
def orb_main_loop():
    print(f"🚀 Starting final_orb_v2 — Live (mainnet futures data) + Testnet bracket orders")
    print(f"SYMBOL: {symbol} | Base USDT: {BASE_USDT} | Leverage: {LEVERAGE}x | DEMO_MODE: {DEMO_MODE}")
    print()

    while True:
        now = datetime.now(IST)

        # Reset logic
        if now.hour == RESET_HOUR and now.minute < 5:
            print(f"[{print_ist_time()}] 🔄 Daily reset triggered — closing all open trades...")
            try:
                client_test.futures_cancel_all_open_orders(symbol=binance_symbol)
                client_test.futures_create_order(
                    symbol=binance_symbol,
                    side=SIDE_SELL,
                    type=ORDER_TYPE_MARKET,
                    quantity=0.1,
                    reduceOnly=True
                )
            except Exception as e:
                print(f"⚠️ Reset error: {e}")

        # Run only every 5 minutes (aligned with system time)
        if now.minute % 5 == 0 and now.second < 5:
            df = fetch_futures_klines(symbol=binance_symbol, interval="1m", lookback="12 hours ago UTC")
            or_high, or_low, or_mid = get_opening_range(df)
            if or_high is None:
                print("⚠️ OR not ready yet...")
                time.sleep(60)
                continue

            print(f"\n📊 Opening Range (IST): High {or_high:.2f} | Low {or_low:.2f} | Mid {or_mid:.2f}")
            df_trades = reconstruct_trades(df, or_high, or_low, or_mid)
            if not df_trades.empty:
                print(df_trades.tail(3))
            else:
                print("No trades reconstructed yet.")

            # Check for entries
            mark_price = float(client_main.futures_mark_price(symbol=binance_symbol)['markPrice'])
            range_ = or_high - or_low
            qty = round_to_precision((BASE_USDT * LEVERAGE) / mark_price, QTY_PRECISION)

            if mark_price > or_high:
                targets = [or_high + (i+1)*range_ for i in range(5)]
                print(f"\n[{print_ist_time()}] | 📈 BUY triggered @ {mark_price:.2f} | SL {or_mid:.2f}")
                place_bracket_orders(mark_price, or_mid, targets + [None], qty, side="BUY")

            elif mark_price < or_low:
                targets = [or_low - (i+1)*range_ for i in range(5)]
                print(f"\n[{print_ist_time()}] | 📉 SELL triggered @ {mark_price:.2f} | SL {or_mid:.2f}")
                place_bracket_orders(mark_price, or_mid, targets + [None], qty, side="SELL")

        time.sleep(3)

# -----------------------------------------------
# START
# -----------------------------------------------
if __name__ == "__main__":
    orb_main_loop()


🚀 Starting final_orb_v2 — Live (mainnet futures data) + Testnet bracket orders
SYMBOL: BTCUSDT_PERP | Base USDT: 1000 | Leverage: 20x | DEMO_MODE: False


📊 Opening Range (IST): High 110516.00 | Low 110230.00 | Mid 110373.00
  side  entry_price          entry_time        sl    qty  \
2  BUY     110559.6 2025-11-02 09:46:00  110373.0  0.182   
3  BUY     110517.2 2025-11-02 11:01:00  110373.0  0.182   
4  BUY     110556.5 2025-11-02 14:03:00  110373.0  0.182   

                                parts  \
2  [0.15, 0.2, 0.25, 0.15, 0.15, 0.1]   
3  [0.15, 0.2, 0.25, 0.15, 0.15, 0.1]   
4  [0.15, 0.2, 0.25, 0.15, 0.15, 0.1]   

                                             targets  pnl exit_price  \
2  [110802.0, 111088.0, 111374.0, 111660.0, 11194...  0.0       None   
3  [110802.0, 111088.0, 111374.0, 111660.0, 11194...  0.0       None   
4  [110802.0, 111088.0, 111374.0, 111660.0, 11194... -0.0       None   

   exit_time              reason pnl_value  
2   110802.0 2025-11-02 10:14:00   

KeyboardInterrupt: 